# Imports

In [ ]:
!pip install statannotations

In [ ]:
import os

import sys, importlib

import logging
import warnings
import time
from copy import deepcopy
from typing import Tuple


import numpy as np
import xarray as xr
import pandas as pd

import math
from scipy.optimize import curve_fit
from scipy.interpolate import griddata

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import seaborn as sns

import skimage.measure
import scipy

from statannotations.Annotator import Annotator

In [ ]:
from utils import circular_mean, circular_mean_length, circular_variance, circular_stddev
from restore_data import recover_data

In [ ]:
log = logging.getLogger(__name__)

import matplotlib.font_manager

from IPython.display import set_matplotlib_formats
%matplotlib inline

In [ ]:
plt.style.context('seaborn-paper')
set_matplotlib_formats('svg')
plt.rcParams.update(plt.rcParamsDefault)

linewidth = 0.25
titlesize = 'medium'
labelsize = 'small'
ticksize = 'x-small'

markersize = 10
scattersize = 5

palette_stages='gist_heat'

plt.rcParams.update({
    'figure.figsize': [5.9/2, 5.9/2 * 2/3],

    'text.usetex': False,
    'font.size': 10,
    'font.family': 'sans-serif',
    # 'font.sans-serif': 'Helvetica',

    'figure.titlesize': titlesize,
    'legend.title_fontsize': labelsize,
    'legend.fontsize': ticksize,
    'axes.labelsize': labelsize,
    'xtick.labelsize': ticksize,
    'ytick.labelsize': ticksize,
    'ytick.labelsize': ticksize,

    'figure.autolayout': False,

    'axes.linewidth': linewidth,
    'xtick.major.width': linewidth,
    'xtick.minor.width': 0.8*linewidth,
    'ytick.major.width': linewidth,
    'ytick.minor.width': 0.8*linewidth,
    'grid.linewidth': linewidth,

    'patch.linewidth': linewidth,

    'lines.markersize': scattersize
})

In [ ]:
from IPython.display import set_matplotlib_formats
%matplotlib inline

In [ ]:

from matplotlib import font_manager
matplotlib.font_manager.findSystemFonts(fontpaths='/usr/share/fonts/truetype', fontext='ttf')
font_manager.findfont('Helvetica', fallback_to_default=False)

In [ ]:
basepath = "/home/julian/ownCloud/Institution/Collaboration_NCBS_UNIGE"
data_basedir = "data"
rel_paths=[
    'HCA-Actn4_new/E8',
    'HCA-Actn4_new/E10',
    'HCA-Actn4_new/E12',
    'HCA-Actn4/E8',
    'HCA-Actn4/E10',
    'HCA-Actn4/E12',
    'HCA-Actn4/E14',
    'Vangl2_KO/Control',
    'Vangl2_KO/Treatment',
    'Vangl2_KO/Treatment_Vangl2',
    'Atoh1/Control',
    'Atoh1/Treatment',
    'MLCK_inhibition/DMSO__4hrs',
    'MLCK_inhibition/DMSO__16hrs',
    'MLCK_inhibition/ML7__4hrs',
    'MLCK_inhibition/ML7__16hrs',
    'rock_inhibitions',
    'PTX_inhibitions'
]
data_path = "102_treated_data"
save_file_prefix='summary'

In [ ]:
figure_path = 'analysis/1_figures'
figure_path = os.path.abspath(os.path.join(basepath, figure_path))

In [ ]:
lineplot_kwargs = dict(
    markers=True,
    ls='',
    dashes=False,
    ci='sd',
    err_style='bars'
)

position_ordering = ["0", "25I", "25S", "50I", "50S", "75I", "75S", "100"]
# stage_ordering = ["E8", "E10", "E12", "E13", "E14"]
stage_ordering = ["E8", "E10", "E12", "E14"]

experiment_order=[
    'MLCKdmso-4hrs',
    'MLCKdmso-16hrs',
    'MLCKml7-4hrs',
    'MLCKml7-16hrs',
    'Vangl2Control',
    'Vangl2KO',
    'Vangl2KO-dense',
    'Live0Control',
    'Live150Control',
    'Live480Control',
    'Live0ML7',
    'Live150ML7',
    'Live400ML7',
]

treatment_order=[
    'MLCKdmso',
    'MLCKml7',
    'Vangl2Control',
    'Vangl2KO',
    'LiveControl',
    'LiveML7',
]

lineplot_position_stage_kwargs = dict(
    style='position',
    style_order=position_ordering,
    hue='stage',
    hue_order=stage_ordering,
    size='sample_id',
    palette=palette_stages
)
lineplot_position_stage_kwargs.update(lineplot_kwargs)

In [ ]:
paths = []
for p in rel_paths:
    paths.append(os.path.join(basepath, data_basedir, p, data_path))
cells, bonds = recover_data(paths=paths, save_file_prefix=save_file_prefix)
cells = cells.loc[~cells['is_border_cell']]
if not 'aligned_with_PD_axis' in cells.columns:
    cells['aligned_with_PD_axis'] = False
    bonds['aligned_with_PD_axis'] = False
else:
    cells['aligned_with_PD_axis'] = np.where(
        ~np.isnan(np.asarray(cells['aligned_with_PD_axis'], dtype=float)),
        cells['aligned_with_PD_axis'],
        False
    )
    bonds['aligned_with_PD_axis'] = np.where(
        ~np.isnan(np.asarray(bonds['aligned_with_PD_axis'], dtype=float)),
        bonds['aligned_with_PD_axis'],
        False
    )

if 'Experiment' in cells.columns:
    cells['Experiment'] = cells['Experiment'].where(
        ~cells['Experiment'].isnull(),
        'Control'
    )
    bonds['Experiment'] = bonds['Experiment'].where(
        ~bonds['Experiment'].isnull(),
        'Control'
    )
else:
    cells['Experiment'] = 'Control'
    bonds['Experiment'] = 'Control'

# # For live imaging 
# cells.loc[np.isnan(cells['frame']), 'frame'] = 0
# bonds.loc[np.isnan(bonds['frame']), 'frame'] = 0

print(cells['Experiment'].unique())

# Map to treatment
cells['Treatment'] = cells['Experiment']
bonds['Treatment'] = bonds['Experiment']

cells.loc[cells['Experiment']=='Live0Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live150Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live480Control', 'Treatment'] = 'LiveControl'
cells.loc[cells['Experiment']=='Live0ML7', 'Treatment'] = 'LiveML7'
cells.loc[cells['Experiment']=='Live150ML7', 'Treatment'] = 'LiveML7'
cells.loc[cells['Experiment']=='Live400ML7', 'Treatment'] = 'LiveML7'

bonds.loc[bonds['Experiment']=='Live0Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live150Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live480Control', 'Treatment'] = 'LiveControl'
bonds.loc[bonds['Experiment']=='Live0ML7', 'Treatment'] = 'LiveML7'
bonds.loc[bonds['Experiment']=='Live150ML7', 'Treatment'] = 'LiveML7'
bonds.loc[bonds['Experiment']=='Live400ML7', 'Treatment'] = 'LiveML7'

cells['Experiment_Series'] = cells['Treatment']
cells.loc[cells['Treatment']=='Atoh1control', 'Experiment_Series'] = 'Atoh1'
cells.loc[cells['Treatment']=='Atoh1KO', 'Experiment_Series'] = 'Atoh1'
cells.loc[cells['Treatment']=='MLCKdmso-4hrs', 'Experiment_Series'] = 'MLCK'
cells.loc[cells['Treatment']=='MLCKml7-4hrs', 'Experiment_Series'] = 'MLCK'
cells.loc[cells['Treatment']=='MLCKdmso-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
cells.loc[cells['Treatment']=='MLCKml7-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
cells.loc[cells['Treatment']=='ROCKcontrol-4hrs', 'Experiment_Series'] = 'ROCK'
cells.loc[cells['Treatment']=='ROCKy27632-4hrs', 'Experiment_Series'] = 'ROCK'
cells.loc[cells['Treatment']=='ROCKcontrol-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
cells.loc[cells['Treatment']=='ROCKy27632-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
cells.loc[cells['Treatment']=='PTXcontrol', 'Experiment_Series'] = 'PTX'
cells.loc[cells['Treatment']=='PTXinhibition', 'Experiment_Series'] = 'PTX'
cells.loc[cells['Treatment']=='Vangl2Control', 'Experiment_Series'] = 'Vangl2'
cells.loc[cells['Treatment']=='Vangl2KO', 'Experiment_Series'] = 'Vangl2'
cells.loc[cells['Treatment']=='LiveControl', 'Experiment_Series'] = 'Live'
cells.loc[cells['Treatment']=='LiveML7', 'Experiment_Series'] = 'Live'

bonds['Experiment_Series'] = bonds['Treatment']
bonds.loc[bonds['Treatment']=='Atoh1control', 'Experiment_Series'] = 'Atoh1'
bonds.loc[bonds['Treatment']=='Atoh1KO', 'Experiment_Series'] = 'Atoh1'
bonds.loc[bonds['Treatment']=='MLCKdmso-4hrs', 'Experiment_Series'] = 'MLCK'
bonds.loc[bonds['Treatment']=='MLCKml7-4hrs', 'Experiment_Series'] = 'MLCK'
bonds.loc[bonds['Treatment']=='MLCKdmso-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
bonds.loc[bonds['Treatment']=='MLCKml7-16hrs', 'Experiment_Series'] = 'MLCK_16hrs'
bonds.loc[bonds['Treatment']=='ROCKcontrol-4hrs', 'Experiment_Series'] = 'ROCK'
bonds.loc[bonds['Treatment']=='ROCKy27632-4hrs', 'Experiment_Series'] = 'ROCK'
bonds.loc[bonds['Treatment']=='ROCKcontrol-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
bonds.loc[bonds['Treatment']=='ROCKy27632-16hrs', 'Experiment_Series'] = 'ROCK_16hrs'
bonds.loc[bonds['Treatment']=='PTXcontrol', 'Experiment_Series'] = 'PTX'
bonds.loc[bonds['Treatment']=='PTXinhibition', 'Experiment_Series'] = 'PTX'
bonds.loc[bonds['Treatment']=='Vangl2Control', 'Experiment_Series'] = 'Vangl2'
bonds.loc[bonds['Treatment']=='Vangl2KO', 'Experiment_Series'] = 'Vangl2'
bonds.loc[bonds['Treatment']=='LiveControl', 'Experiment_Series'] = 'Live'
bonds.loc[bonds['Treatment']=='LiveML7', 'Experiment_Series'] = 'Live'


# Assign is_control
cells['is_Control'] = False
cells.loc[cells['Treatment']=='Control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Atoh1control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Atoh1KO', 'is_Control'] = False
cells.loc[cells['Treatment']=='MLCKdmso-4hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='MLCKml7-4hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='MLCKdmso-16hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='MLCKml7-16hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='ROCKcontrol-4hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='ROCKy27632-4hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='ROCKcontrol-16hrs', 'is_Control'] = True
cells.loc[cells['Treatment']=='ROCKy27632-16hrs', 'is_Control'] = False
cells.loc[cells['Treatment']=='PTXcontrol', 'is_Control'] = True
cells.loc[cells['Treatment']=='PTXinhibition', 'is_Control'] = False
cells.loc[cells['Treatment']=='Vangl2Control', 'is_Control'] = True
cells.loc[cells['Treatment']=='Vangl2KO', 'is_Control'] = False
cells.loc[cells['Treatment']=='LiveControl', 'is_Control'] = True
cells.loc[cells['Treatment']=='LiveML7', 'is_Control'] = False

bonds['is_Control'] = False
bonds.loc[bonds['Treatment']=='Control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Atoh1control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Atoh1KO', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='MLCKdmso-4hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='MLCKml7-4hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='MLCKdmso-16hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='MLCKml7-16hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='ROCKcontrol-4hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='ROCKy27632-4hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='ROCKcontrol-16hrs', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='ROCKy27632-16hrs', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='PTXcontrol', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='PTXinhibition', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='Vangl2Control', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='Vangl2KO', 'is_Control'] = False
bonds.loc[bonds['Treatment']=='LiveControl', 'is_Control'] = True
bonds.loc[bonds['Treatment']=='LiveML7', 'is_Control'] = False



# correct orientations
print("WARNING reorienting cilia orientations, but not other orientations")
mask_rotate90 = (
    (cells['filename'] == 'MLCKdmso-4hrs_2.tif')
)
cells.loc[mask_rotate90, 'cilium_phi'] -= math.pi/2
cells.loc[mask_rotate90, 'cilium_phi_corrected'] -= math.pi/2

mask_flip_SI = (
    (cells['Experiment'] == 'MLCKdmso-4hrs') |
    (
        (cells['Experiment'] == 'Control') &
        (cells['stage'] == 'E14') &
        (cells['position'] == '75I')
    )
)
print(mask_flip_SI.sum())
cells.loc[mask_flip_SI, 'cilium_phi'] *= -1
cells.loc[mask_flip_SI, 'cilium_phi_corrected'] *= -1


# correct y-axis pointing down
cells['cilium_phi'] *= -1
cells['cilium_phi_corrected'] *= -1


cells['cilium_DX'] = cells['cilium_rho'] * np.cos(cells['cilium_phi'])
cells['cilium_DY'] = cells['cilium_rho'] * np.sin(cells['cilium_phi'])
cells['cilium_DX_corrected'] = cells['cilium_rho_corrected'] * np.cos(cells['cilium_phi_corrected'])
cells['cilium_DY_corrected'] = cells['cilium_rho_corrected'] * np.sin(cells['cilium_phi_corrected'])




cells_experiments = cells
bonds_experiments = bonds

cells = cells.loc[cells['Experiment'] == 'Control']
bonds = bonds.loc[bonds['Experiment'] == 'Control']

# select subsets
HC = cells.loc[cells['is_HC']]
SC = cells.loc[~cells['is_HC']]

In [ ]:
def crop_image(frame, threshold_HC):
    if frame['is_HC'].count() < threshold_HC:
        return 0
    
    L = frame['center_x_cells'].max()
    H = frame['center_y_cells'].max()

    return  1 + (frame['center_x_cells'] < L / 2) + 2 * (frame['center_y_cells'] < H / 2)

### correct neighbourhood

In [ ]:
HC = cells.loc[cells['is_HC']]
SC = cells.loc[~cells['is_HC']]
m_HC = np.polyfit(HC['normalized_area_cells'], HC['num_neighbors'], 1)[0]
m_SC = np.polyfit(SC['normalized_area_cells'], SC['num_neighbors'], 1)[0]
cells['num_neighbors__corr'] = np.where(
    cells['is_HC'],
    cells['num_neighbors'] + m_HC * (1 - cells['normalized_area_cells']),
    cells['num_neighbors'] + m_SC * (1 - cells['normalized_area_cells']),
)
cells_experiments['num_neighbors__corr'] = np.where(
    cells_experiments['is_HC'],
    cells_experiments['num_neighbors'] + m_HC * (1 - cells_experiments['normalized_area_cells']),
    cells_experiments['num_neighbors'] + m_SC * (1 - cells_experiments['normalized_area_cells']),
)

### Align tissue axes

In [ ]:
polarity_data = cells_experiments.copy()

results = pd.DataFrame()
grp=['Experiment_Series', 'stage', 'position', 'is_Control', 'filename', 'file_id']
results['mean_orientation'] = polarity_data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
results['circular_stddev'] = polarity_data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
results['circular_variance'] = polarity_data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_variance, eulerian=True)
results['resulting_length'] = polarity_data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True)
results['datapoints'] = polarity_data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

results['mean_orientation__angles'] = (results['mean_orientation'] / math.pi * 180 + 360) % 360
results['circular_stddev__angles'] = (results['circular_stddev'] / math.pi * 180 + 360) % 360

results.to_csv("{}/{}/{}_polarity.csv".format(basepath, data_basedir, save_file_prefix))
results = results.reset_index()


results_global_mean = pd.DataFrame()
grp_global_mean=grp[:-2]
results_global_mean['mean_orientation'] = polarity_data.groupby(by=grp_global_mean)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
results_global_mean['circular_stddev'] = polarity_data.groupby(by=grp_global_mean)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
results_global_mean['circular_variance'] = polarity_data.groupby(by=grp_global_mean)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_variance, eulerian=True)
results_global_mean['resulting_length'] = polarity_data.groupby(by=grp_global_mean)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True)
results_global_mean['datapoints'] = polarity_data.groupby(by=grp_global_mean)['cilium_DX_corrected'].apply(lambda d: d.count())

results_global_mean['mean_orientation__angles'] = (results_global_mean['mean_orientation'] / math.pi * 180 + 360) % 360
results_global_mean['circular_stddev__angles'] = (results_global_mean['circular_stddev'] / math.pi * 180 + 360) % 360

results_global_mean.to_csv("{}/{}/{}_polarity__global_mean.csv".format(basepath, data_basedir, save_file_prefix))
results_global_mean = results_global_mean.reset_index()



polarity_data['cilium_phi_corrected__aligned'] = polarity_data['cilium_phi_corrected'].copy()
for file_id in polarity_data['file_id'].unique():
    polarity_data.loc[polarity_data['file_id']==file_id, 'cilium_phi_corrected__aligned'] -= results.loc[results['file_id']==file_id, 'mean_orientation'].iloc[0]

for series in results['Experiment_Series'].unique():
    for is_Control in results['is_Control'].unique():
        mask = (
            (results['Experiment_Series']==series) &
            (results['is_Control']==is_Control)
        )

        cmean = circular_mean( results.loc[mask, ['resulting_length', 'mean_orientation']], eulerian=False)

        mask = (
            (polarity_data['Experiment_Series']==series) &
            (polarity_data['is_Control']==is_Control)
        )
        polarity_data.loc[mask, 'cilium_phi_corrected__aligned'] += cmean

polarity_data['cilium_DX_corrected__aligned'] = polarity_data['cilium_rho_corrected'] * np.cos(polarity_data['cilium_phi_corrected__aligned'])
polarity_data['cilium_DY_corrected__aligned'] = polarity_data['cilium_rho_corrected'] * np.sin(polarity_data['cilium_phi_corrected__aligned'])

results_aligned = pd.DataFrame()
grp_aligned=grp[:-2]
results_aligned['mean_orientation'] = polarity_data.groupby(by=grp_aligned)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_mean, eulerian=True)
results_aligned['circular_stddev'] = polarity_data.groupby(by=grp_aligned)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_stddev, eulerian=True)
results_aligned['circular_variance'] = polarity_data.groupby(by=grp_aligned)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_variance, eulerian=True)
results_aligned['resulting_length'] = polarity_data.groupby(by=grp_aligned)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_mean_length, eulerian=True)
results_aligned['datapoints'] = polarity_data.groupby(by=grp_aligned)['cilium_DX_corrected'].apply(lambda d: d.count())

results_aligned['mean_orientation__angles'] = (results_aligned['mean_orientation'] / math.pi * 180 + 360) % 360
results_aligned['circular_stddev__angles'] = (results_aligned['circular_stddev'] / math.pi * 180 + 360) % 360
results_aligned.to_csv("{}/{}/{}_polarity__aligned.csv".format(basepath, data_basedir, save_file_prefix))
results_aligned = results_aligned.reset_index()

cells_experiments['cilium_phi_corrected__aligned'] = polarity_data['cilium_phi_corrected__aligned'].copy()
cells_experiments['cilium_DX_corrected__aligned'] = polarity_data['cilium_DX_corrected__aligned'].copy()
cells_experiments['cilium_DY_corrected__aligned'] = polarity_data['cilium_DY_corrected__aligned'].copy()

In [ ]:
def tissue_averages(cells, bonds, tissue_grp):
    cells = cells.loc[~cells['is_border_cell']]
    HC = cells.loc[cells['is_HC']]
    SC = cells.loc[~cells['is_HC']]

    print(cells['Experiment'].unique())
    
    tissue_data = pd.DataFrame()


    # count cells
    tissue_data['num_HC'] = HC.groupby(by=tissue_grp)['is_HC'].count()
    tissue_data['num_SC'] = SC.groupby(by=tissue_grp)['is_HC'].count()
    tissue_data['num_cells'] = cells.groupby(by=tissue_grp)['is_HC'].count()

    tissue_data['HC_density'] = tissue_data['num_HC'] / tissue_data['num_cells']


    # cell areas
    tissue_data['HC_area'] = HC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    tissue_data['HC_area__std'] = HC.groupby(by=tissue_grp)['normalized_area_cells'].std()

    tissue_data['SC_area'] = SC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    tissue_data['SC_area__std'] = SC.groupby(by=tissue_grp)['normalized_area_cells'].std()

    tissue_data['area__no_norm'] = cells.groupby(by=tissue_grp)['area'].mean()
    tissue_data['area__no_norm__std'] = cells.groupby(by=tissue_grp)['area'].std()

    tissue_data['HC_area__no_norm'] = HC.groupby(by=tissue_grp)['area'].mean()
    tissue_data['HC_area__no_norm__std'] = HC.groupby(by=tissue_grp)['area'].std()

    tissue_data['SC_area__no_norm'] = SC.groupby(by=tissue_grp)['area'].mean()
    tissue_data['SC_area__no_norm__std'] = SC.groupby(by=tissue_grp)['area'].std()


    tissue_data['relative_area'] = (
        HC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
        / SC.groupby(by=tissue_grp)['normalized_area_cells'].mean()
    )
    tissue_data['relative_area__std'] = tissue_data['relative_area'] * abs(
        (  HC.groupby(by=tissue_grp)['normalized_area_cells'].std()
        / HC.groupby(by=tissue_grp)['normalized_area_cells'].mean())
        -
        (  SC.groupby(by=tissue_grp)['normalized_area_cells'].std()
        / SC.groupby(by=tissue_grp)['normalized_area_cells'].mean())
    )

    tissue_data['HC_area_coverage'] = tissue_data['HC_area'] * tissue_data['HC_density']


    # neighborhood
    tissue_data['neighbor_number'] = cells.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['neighbor_number__std'] = cells.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['neighbor_number__corr'] = cells.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['neighbor_number__corr__std'] = cells.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['coordination_number'] = HC.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['coordination_number__std'] = HC.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['coordination_number__corr'] = HC.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['coordination_number__corr__std'] = HC.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['subordination_number'] = SC.groupby(by=tissue_grp)['num_neighbors'].mean()
    tissue_data['subordination_number__std'] = SC.groupby(by=tissue_grp)['num_neighbors'].std()
    tissue_data['subordination_number__corr'] = SC.groupby(by=tissue_grp)['num_neighbors__corr'].mean()
    tissue_data['subordination_number__corr__std'] = SC.groupby(by=tissue_grp)['num_neighbors__corr'].std()

    tissue_data['HH_contacts'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].mean()
    tissue_data['HH_contacts__std'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].std()
    tissue_data['HH_contacts_total'] = HC.groupby(by=tissue_grp)['num_hair_neighbors'].sum()

    tissue_data['SH_contacts'] = SC.groupby(by=tissue_grp)['num_hair_neighbors'].mean()
    tissue_data['SH_contacts__std'] = SC.groupby(by=tissue_grp)['num_hair_neighbors'].std()

    tissue_data['num_HH_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'HH').sum()), dtype=int)
    tissue_data['num_HS_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'HS').sum()), dtype=int)
    tissue_data['num_SS_bonds'] = np.asarray(bonds.groupby(by=tissue_grp).apply(lambda bonds: (bonds['type'] == 'SS').sum()), dtype=int)
    tissue_data['num_HC_bonds'] = tissue_data['num_HH_bonds'] + tissue_data['num_HS_bonds']
    tissue_data['num_bonds'] = tissue_data['num_HH_bonds'] + tissue_data['num_HS_bonds'] + tissue_data['num_SS_bonds']
    tissue_data['fraction_HH_bonds'] = tissue_data['num_HH_bonds'] / tissue_data['num_bonds']


    # spatial organisation
    tissue_data['hexatic_order'] = HC.groupby(by=tissue_grp)['hexatic_order'].mean()
    tissue_data['hexatic_order__std'] = HC.groupby(by=tissue_grp)['hexatic_order'].std()
    tissue_data['hexatic_order_corrected'] = HC.groupby(by=tissue_grp)['hexatic_order_corrected'].mean()
    tissue_data['hexatic_order_corrected__std'] = HC.groupby(by=tissue_grp)['hexatic_order_corrected'].std()
    tissue_data['hexatic_order_ellipse_fit'] = HC.groupby(by=tissue_grp)['hexatic_order_ellipse_fit'].mean()
    tissue_data['hexatic_order_ellipse_fit__std'] = HC.groupby(by=tissue_grp)['hexatic_order_ellipse_fit'].std()

    tissue_data['num_next_HC_neighbors'] = HC.groupby(by=tissue_grp)['num_next_HC_neighbors'].mean()
    tissue_data['num_next_HC_neighbors__std'] = HC.groupby(by=tissue_grp)['num_next_HC_neighbors'].std()


    # polarity
    tissue_data['num_cilia'] = cells.groupby(by=tissue_grp)['cilium_DX'].count()

    tissue_data['polarity'] = HC.groupby(by=tissue_grp)[['cilium_DX', 'cilium_DY']].apply(circular_mean)
    tissue_data['polarity_angles'] = tissue_data['polarity'] / math.pi * 180.
    tissue_data['polarity__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX', 'cilium_DY']].apply(circular_variance)

    HC['cilium_DX_corrected'] = HC['cilium_rho_corrected'] * np.cos(HC['cilium_phi_corrected'])
    HC['cilium_DY_corrected'] = HC['cilium_rho_corrected'] * np.sin(HC['cilium_phi_corrected'])
    tissue_data['polarity_corrected'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean)
    tissue_data['polarity_corrected_angles'] = tissue_data['polarity_corrected'] / math.pi * 180.
    tissue_data['polarity_corrected__stddev'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev)
    tissue_data['polarity_corrected__stddev_angles'] = tissue_data['polarity_corrected__stddev'] / math.pi * 180
    tissue_data['polarity_corrected__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_variance)

    if 'cilium_DX_corrected__aligned' in tissue_data.columns:
        tissue_data['polarity_corrected_aligned'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_mean)
        tissue_data['polarity_corrected_aligned_angles'] = tissue_data['polarity_corrected_aligned'] / 180 * math.pi
        tissue_data['polarity_corrected_aligned__stddev'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_stddev)
        tissue_data['polarity_corrected_aligned__stddev_angles'] = np.where(
            tissue_data['num_cilia'] >= 10,
            tissue_data['polarity_corrected_aligned__stddev'] / math.pi * 180,
            np.nan
        )
        tissue_data['polarity_corrected_aligned__variance'] = HC.groupby(by=tissue_grp)[['cilium_DX_corrected__aligned', 'cilium_DY_corrected__aligned']].apply(circular_variance)

    mask = tissue_data['num_cilia'] < 10
    tissue_data.loc[mask, 'polarity'] = np.nan
    tissue_data.loc[mask, 'polarity__variance'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected__variance'] = np.nan
    tissue_data.loc[mask, 'polarity_corrected__stddev'] = np.nan


    return tissue_data.reset_index()

In [ ]:
__tissue_grp = [
    'Experiment',
    'Experiment_Series',
    'Treatment',
    'is_Control',
    'aligned_with_PD_axis',
    'stage',
    'Stage',
    'position',
    'SI_position',
    'S-I position',
    'SI_position_long',
    'PD_position',
    'P-D position',
]

tissue_grp = [
    'file_id',
    'sample_id',
    # 'crop_id',
    'filename',
    'filepath',
]

for g in __tissue_grp:
    tissue_grp.append(g)

In [ ]:
tissue_data = tissue_averages(cells, bonds, tissue_grp)

In [ ]:
__tissue_data = tissue_averages(cells_experiments, bonds_experiments, tissue_grp)

In [ ]:
cells.to_csv("{}/{}/{}_cells.csv".format(basepath, data_basedir, save_file_prefix))
bonds.to_csv("{}/{}/{}_bonds.csv".format(basepath, data_basedir, save_file_prefix))

cells_experiments.to_csv("{}/{}/{}_cells_w_treatments.csv".format(basepath, data_basedir, save_file_prefix))
bonds_experiments.to_csv("{}/{}/{}_bonds_w_treatments.csv".format(basepath, data_basedir, save_file_prefix))

tissue_data.to_csv("{}/{}/{}_tissue_averages.csv".format(basepath, data_basedir, save_file_prefix))
__tissue_data.to_csv("{}/{}/{}_tissue_averages_w_treatments.csv".format(basepath, data_basedir, save_file_prefix))

tissue_averages(cells_experiments, bonds_experiments, __tissue_grp).to_csv("{}/{}/{}_averages.csv".format(basepath, data_basedir, save_file_prefix))

print("Number of data-points")
data_information = pd.DataFrame()
data_information['cells'] = cells.groupby(by=['stage', 'position'])[['num_neighbors']].count()
data_information['HC (cilia)'] = cells.loc[cells['is_HC']].groupby(by=['stage', 'position']).apply(lambda hc: (hc['num_neighbors'].count(), hc['cilium_DX'].count()))
data_information['SC'] = cells.loc[~cells['is_HC']].groupby(by=['stage', 'position'])[['num_neighbors']].count()
data_information['tissues (w. cilia)'] = cells.loc[cells['is_HC']].groupby(by=['stage', 'position']).apply(lambda hc: (len(np.unique(hc['file_id'])), len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id']))))

print(data_information.to_string())
data_information.to_csv("{}/{}/{}_cells_infos.csv".format(basepath, data_basedir, save_file_prefix))


print("\nNumber of data-points with known P-D axis")
data_information_with_orientation = pd.DataFrame()
__cells_with_axis = cells.loc[cells['aligned_with_PD_axis']]
data_information_with_orientation['cells'] = __cells_with_axis.groupby(by=['stage', 'position'])[['num_neighbors']].count()
data_information_with_orientation['HC (cilia)'] = __cells_with_axis.loc[__cells_with_axis['is_HC']].groupby(by=['stage', 'position']).apply(lambda hc: (hc['num_neighbors'].count(), hc['cilium_DX'].count()))
data_information_with_orientation['SC'] = __cells_with_axis.loc[~__cells_with_axis['is_HC']].groupby(by=['stage', 'position'])[['num_neighbors']].count()
data_information_with_orientation['tissues (w. cilia)'] = __cells_with_axis.loc[__cells_with_axis['is_HC']].groupby(by=['stage', 'position']).apply(lambda hc: (len(np.unique(hc['file_id'])), len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id']))))

print(data_information_with_orientation.to_string())
data_information_with_orientation.to_csv("{}/{}/{}_cells_aligned_infos.csv".format(basepath, data_basedir, save_file_prefix))


print("\nNumber of data-points treatments")
data_information_treatments = pd.DataFrame()
__cells_w_treatment = cells_experiments.loc[(cells_experiments['position'] == '25I') & (cells_experiments['stage'] == 'E10')]
data_information_treatments['cells'] = __cells_w_treatment.groupby(by=['Experiment', 'stage', 'position'])[['num_neighbors']].count()
data_information_treatments['HC (cilia)'] = __cells_w_treatment.loc[__cells_w_treatment['is_HC']].groupby(by=['Experiment', 'stage', 'position']).apply(lambda hc: (hc['num_neighbors'].count(), hc['cilium_DX'].count()))
data_information_treatments['SC'] = __cells_w_treatment.loc[~__cells_w_treatment['is_HC']].groupby(by=['Experiment', 'stage', 'position'])[['num_neighbors']].count()
data_information_treatments['tissues (w. cilia)'] = __cells_w_treatment.loc[__cells_w_treatment['is_HC']].groupby(by=['Experiment', 'stage', 'position']).apply(lambda hc: (len(np.unique(hc['file_id'])), len(np.unique(hc.loc[~np.isnan(hc['cilium_DX']), 'file_id']))))

print(data_information_treatments.to_string())
data_information_treatments.to_csv("{}/{}/{}_cells_w_treatments_infos.csv".format(basepath, data_basedir, save_file_prefix))


In [ ]:
print(__tissue_data.columns)

In [ ]:
print(cells.columns)

In [ ]:
data = pd.DataFrame()
data['Series'] = __tissue_data['Experiment_Series']
data['Experiment'] = __tissue_data['Experiment']
data['Experiment'] = __tissue_data['Experiment']
data['Name of the experiment'] = __tissue_data.apply(lambda data: f"{data['stage']} {data['position']}" if data['Experiment_Series']=='Control' else data['Experiment'], axis=1)
data['Number of HCs (cilia) / SCs examined'] = __tissue_data.apply(lambda data: f"{data['num_HC']} ({data['num_cilia']}) / {data['num_SC']}", axis=1)
data['Surface area of HCs'] = __tissue_data['HC_area__no_norm']
data['Surface area of SCs'] = __tissue_data['SC_area__no_norm']
data['Average HC neighbour number'] = __tissue_data['coordination_number']
data['Corrected HC neighbour number'] = __tissue_data['coordination_number__corr']
data['Average hexatic order (corrected)'] = __tissue_data['hexatic_order_corrected']
data['Mean orientation'] = __tissue_data['polarity_corrected_angles']
data['CSD'] = __tissue_data['polarity_corrected__stddev_angles']

def rename_experiment(name):
    if name == 'Vangl2Control':
        return 'Vangl2 Control'
    if name == 'Vangl2KO':
        return 'Vangl2 KO'
    if name == 'Atoh1control':
        return 'Atoh1 Control'
    if name == 'Atoh1KO':
        return 'Atoh1 KO'
    if name == 'MLCKdmso-4hrs':
        return 'DMSO 4 hrs'
    if name == 'MLCKdmso-16hrs':
        return 'DMSO 16 hrs'
    if name == 'MLCKml7-4hrs':
        return 'ML7 4 hrs'
    if name == 'MLCKml7-16hrs':
        return 'ML7 4 + 12 hrs'
    if name == 'ROCKcontrol-4hrs':
        return 'DMEM 4 hrs'
    if name == 'ROCKcontrol-16hrs':
        return 'DMEM 16 hrs'
    if name == 'ROCKy27632-4hrs':
        return 'Y-27632 4 hrs'
    if name == 'ROCKy27632-16hrs':
        return 'Y-27632 4 + 12 hrs'
    if name == 'PTXcontrol':
        return 'DMEM 4 hrs (PTX control)'
    if name == 'PTXinhibition':
        return 'PTX 4 hrs'
    else:
        return name
data['Name of the experiment'] = data['Name of the experiment'].apply(rename_experiment)

data.loc[data['Series']=='Control'].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_BP.csv")
data.loc[(data['Series']=='Control') & ((__tissue_data['position']=='25I') | (__tissue_data['position']=='75I'))].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_BP_25I_75I.csv")
data.loc[data['Series']!='Control'].to_csv(f"{basepath}/{data_basedir}/{save_file_prefix}_tissue_averages_publication_treatments.csv")

In [ ]:
# __export = [
#     'sample_id',
#     'HC_area',
#     'HC_area__no_norm',
#     'SC_area',
#     'SC_area__no_norm',
#     'relative_area',
#     'HC_density',
#     'neighbor_number',
#     'coordination_number',
#     'subordination_number',
#     'fraction_HH_bonds',
#     'hexatic_order',
#     'hexatic_order_corrected',
#     'num_cells',
#     'num_HC',
#     'num_SC',
#     'polarity_angles',
#     'polarity_corrected_angles',
#     'polarity_corrected__stddev_angles',
#     'num_cilia',
# ]
# export_explants = [
#     'stage',
#     'position'
# ]
# export_treatments  =[
#     'Experiment'
# ]
# for e in __export:
#     export_explants.append(e)
#     export_treatments.append(e)

# tissue_data[export_explants].to_csv("{}/{}/{}_tissue_averages__publication.csv".format(basepath, data_basedir, save_file_prefix))
# __tissue_data.loc[__tissue_data['Experiment']!='Control', export_treatments].to_csv("{}/{}/{}_tissue_averages_w_treatments__publication.csv".format(basepath, data_basedir, save_file_prefix))


In [ ]:
print(tissue_data.groupby(by=['stage', 'PD_position'])[['HC_area', 'HC_density', 'HC_area_coverage']].mean())
# print(tissue_data.groupby(by=['stage', 'PD_position'])[['HC_area', 'HC_density']].mean())
print(tissue_data.groupby(by=['stage', 'PD_position'])[['HC_area', 'HC_density', 'HC_area_coverage']].mean().groupby(by='stage').mean())
print(tissue_data.groupby(by=['stage'])[['hexatic_order_corrected']].mean())
print(tissue_data.groupby(by=['stage'])[['hexatic_order_corrected']].std())
print(tissue_data.groupby(by=['stage', 'PD_position'])[['HC_area', 'HC_density', 'HC_area_coverage']].mean().groupby(by='PD_position').max())
print(tissue_data.groupby(by=['stage'])[['HC_area', 'HC_density', 'HC_area_coverage']].mean())
print(tissue_data.groupby(by=['stage'])[['HC_area', 'HC_density', 'HC_area_coverage']].std())


In [ ]:
print(__tissue_data.groupby(by=['Experiment_Series', 'is_Control', 'stage'])[['hexatic_order_corrected']].mean())
print(__tissue_data.groupby(by=['Experiment_Series', 'is_Control', 'stage'])[['hexatic_order_corrected']].std())

In [ ]:
print("WARNING tissues with more than 5% H-H contacts (false contacts)")
print(__tissue_data.sort_values(by='fraction_HH_bonds', ascending=False)[['filename', 'aligned_with_PD_axis', 'file_id', 'fraction_HH_bonds', 'num_HH_bonds']].to_string())

In [ ]:
print("WARNING tissues with more than 5% H-H contacts (false contacts)")
print(tissue_data[['file_id', 'aligned_with_PD_axis', 'filename', 'HH_contacts', "HH_contacts_total", 'HC_density']].sort_values(by='HH_contacts', ascending=False))

# Schematics

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[3, 3])
phi = np.linspace(0, 2 * math.pi, 101)
long = 3
short = 1
rotate = 30/180*math.pi - math.pi

def __rescale_dx_dy(displ_x, displ_y, *, long, short, theta, scatter=False):
    """Rescale to circle for correction"""
    dx_ = (displ_x * np.cos(-theta) - displ_y * np.sin(-theta)) / (long / short)
    dy_ =  displ_x * np.sin(-theta) + displ_y * np.cos(-theta)

    dx = dx_ * np.cos(theta) - dy_ * np.sin(theta)
    dy = dx_ * np.sin(theta) + dy_ * np.cos(theta)

    rho = (long * short)**0.5

    return dx/short, dy/short

def Rho_ellipse(angle, *, short, long, rotate):
    return long * short / (long**2 * np.sin(angle - rotate)**2 + short**2 * np.cos(angle - rotate)**2)**0.5

# ax.scatter(0, 0, color='black', marker='+', lw=0.5)
# ax.plot(np.linspace(-1.2 * long, 1.2 * long, 21) * np.cos(rotate), np.linspace(-1.2 * long, 1.2 * long, 21) * np.sin(rotate), c='black')

# the circle with equal area
area = math.pi * long * short
rho = (area / math.pi)**0.5
ax.plot(rho * np.cos(phi), rho * np.sin(phi), color='black', linestyle='dashed', lw=0.5)

rho_ellipse = Rho_ellipse(phi, short=short, long=long, rotate=rotate)
ax.plot(rho_ellipse * np.cos(phi), rho_ellipse * np.sin(phi), color='black', lw=0.5)

# cilium_angle = (np.linspace(0, 90, 4))/180 * math.pi + rotate
cilium_angle = (np.array([10]))/180 * math.pi + rotate
cilium_rho = 0.85 * Rho_ellipse(cilium_angle, short=short, long=long, rotate=rotate)
ax.scatter(cilium_rho* np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), color='red', s=20)

dx, dy = __rescale_dx_dy(cilium_rho * np.cos(cilium_angle), cilium_rho * np.sin(cilium_angle), long=long, short=short, theta=rotate, scatter=True)
ax.quiver(np.zeros_like(dx), np.zeros_like(dx), dx*rho, dy*rho, scale=1, angles='xy', scale_units='xy', color='red')

__rho = Rho_ellipse(0, short=short, long=long, rotate=0)
x, y = __rho * np.cos(rotate), __rho * np.sin(rotate)
ax.quiver(x, y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray')
ax.quiver(-x, -y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), scale=1, angles='xy', scale_units='xy', pivot='tip', color='gray')

__rho = Rho_ellipse(90, short=short, long=long, rotate=0)
x, y = -__rho * np.sin(rotate), __rho * np.cos(rotate)
ax.quiver(x, y, 0.9 * x / __rho * (long - rho), 0.9 * y / __rho * (long - rho), scale=1, angles='xy', scale_units='xy', color='gray')
ax.quiver(-x, -y, -0.9 * x / __rho * (long - rho), -0.9 * y / __rho * (long - rho), scale=1, angles='xy', scale_units='xy', color='gray')

ax.set_xlim([-4, 4])
ax.set_ylim([-3, 3])
ax.set_aspect('equal')


ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
# ax.axis('off')

ax.quiver(0, 0, 2.5*long, 0, scale=1, angles='xy', scale_units='xy', color='black', width=0.0025, pivot='mid')
ax.quiver(0, 0, 0, long*1.75, scale=1, angles='xy', scale_units='xy', color='black', width=0.0025, pivot='mid')




plt.savefig(f"{figure_path}/cell_elongation_correction.svg",format='svg')

# Analysis

## Summary

Loaded data in 0.3638577461242676.
Transformed frame data to global data in 0.4795360565185547s
Updated neighborhood in 9.954678773880005s
Updated hair neighborhood in 28.700066804885864s
Assigned types to bonds in 31.536331176757812s
Extended data in 0.2822737693786621s
Successfully manipulated data in 71.31730127334595.

In [ ]:
# check F-actin distribution
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)

print(cells['sum_px_intensity_cells_ch2'].mean())
print(cells['sum_px_intensity_cells_ch2'].max())

ax1.hist(HC['sum_px_intensity_cells_ch2'], 
         align='left', bins=40, range=[0, 1e6])
ax1.set_ylabel("HC")

ax2.hist(SC['sum_px_intensity_cells_ch2'], 
         align='left', bins=40, range=[0, 1e6])
ax2.set_ylabel("SC")

ax3.hist(cells['sum_px_intensity_cells_ch2'],
         align='left', bins=40, range=[0, 1e6])
ax3.set_xlabel("sum HCA intensity")
ax3.set_ylabel("all")

In [ ]:
print("HC density: {:.3f}".format(
    (cells.loc[cells['is_HC']].dropna(how='all')).shape[0] / cells.shape[0]))
print("HC norm average area: {:.3f} pm {:.3f}".format(
    np.mean(HC['normalized_area_cells']),
    np.std(HC['normalized_area_cells'])
))

print('Area covered by HC {:.3f} of BP.'
      ''.format(np.sum(HC['area_cells'])/np.sum(cells['area_cells'])))
print("")

print("Coordination number: {:.3f} pm {:.3f}".format(
    np.mean(HC['num_neighbors']),
    np.std(HC['num_neighbors'])
))
print("Subordination number: {:.3f} pm {:.3f}".format(
    np.mean(SC['num_neighbors']),
    np.std(SC['num_neighbors'])
))
print("Geometry {:.3f}".format(np.mean(cells['num_neighbors'])))

print("")
print("The count of H-H junctions (total cells):")
HC.groupby(['filename']).apply(
    lambda d: print("{}: {} (for {} HC) = {}"
                    "".format(d['filename'].unique()[0],
                              d['num_hair_neighbors'].sum(),
                              d['is_HC'].sum(),
                              d['num_hair_neighbors'].sum() / d['is_HC'].sum())))



In [ ]:
fig, ax = plt.subplots(1, 1)

sns.histplot(cells[cells['is_HC']], x='area_cells',
             hue='position', hue_order=position_ordering)

## Neighbor analysis

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['is_HC']],
    x='stage',
    y='normalized_area_cells',
    hue='position', hue_order=['25I', '75I'],
    # palette=[''],
    join=False,
    ci='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

print(HC.loc[HC['position'] == '25I'].groupby(by='stage')['normalized_area_cells'].mean())

ax.hlines(y=[1], xmin=-1, xmax=3.5, ls=':', color='gray')


leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')


ax.set_ylim([0, 4.5])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('Normalised HC area')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


plt.savefig(f"{figure_path}/HC_area.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['is_HC']],
    x='stage',
    y='num_neighbors',
    hue='position', hue_order=['25I', '75I'],
    # palette=[''],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

# ax.hlines(y=[1], xmin=-1, xmax=3.5, ls=':', color='gray')


leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')


ax.set_ylim([0, 11])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('HC neighbour number')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)


plt.savefig(f"{figure_path}/HC_neighbour_number.svg",format='svg')

In [ ]:
area_increase = pd.DataFrame()

data = cells.loc[cells['is_HC']].copy()

area_increase['HC_area__E10'] = data.loc[data['stage']=='E10'].groupby(by='position')['area'].mean()
area_increase['HC_area__E14'] = data.loc[data['stage']=='E14'].groupby(by='position')['area'].mean()
area_increase['HC_area__E12'] = data.loc[data['stage']=='E12'].groupby(by='position')['area'].mean()
area_increase['HC_area__E8'] = data.loc[data['stage']=='E8'].groupby(by='position')['area'].mean()
area_increase['ration__E14_E8'] = area_increase['HC_area__E14'] / area_increase['HC_area__E8']
area_increase['ration__E12_E10'] = area_increase['HC_area__E12'] / area_increase['HC_area__E10']
area_increase['ration__E12_E8'] = area_increase['HC_area__E12'] / area_increase['HC_area__E8']

print(area_increase.to_string())


area_increase['HC_area__E10'] = tissue_data.loc[tissue_data['stage']=='E10'].groupby(by='position')['HC_area__no_norm'].mean()
area_increase['HC_area__E14'] = tissue_data.loc[tissue_data['stage']=='E14'].groupby(by='position')['HC_area__no_norm'].mean()
area_increase['HC_area__E12'] = tissue_data.loc[tissue_data['stage']=='E12'].groupby(by='position')['HC_area__no_norm'].mean()
area_increase['HC_area__E8'] = tissue_data.loc[tissue_data['stage']=='E8'].groupby(by='position')['HC_area__no_norm'].mean()
area_increase['ration__E14_E8'] = area_increase['HC_area__E14'] / area_increase['HC_area__E8']
area_increase['ration__E12_E10'] = area_increase['HC_area__E12'] / area_increase['HC_area__E10']
area_increase['ration__E12_E8'] = area_increase['HC_area__E12'] / area_increase['HC_area__E8']

print(area_increase.to_string())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position'] == '25I'],
    x='stage',
    y='area',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # s=2,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])

print(HC.loc[HC['position'] == '25I'].groupby(by='stage')['area'].mean())


ax.set_ylim([0, None])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel("Cell area [$\mu m^2$]")

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/HC_area__no_norm.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position'] == '25I'],
    x='stage',
    y='num_neighbors',
    hue='is_HC',
    hue_order=[False, True],
    palette=['gray', 'red'],
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    errwidth=linewidth,
    markers='.',
)
plt.setp(points.collections, sizes=[markersize])


ax.set_ylim([3, None])
ax.set_xlim([-0.5, 3.5])

ax.set_ylabel('Neighbour number')

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['SC', 'HC'], title='')

plt.savefig(f"{figure_path}/num_neighbors.svg",format='svg')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[5.9/2, 5.9/3])

points = sns.pointplot(
    data=cells.loc[cells['position'] == '25I'],
    ax=ax1,
    x='stage',
    y='area',
    hue='is_HC',
    hue_order=[True, False],
    palette=['gray', 'red'],
    join=False,
    ci='sd',
    # size='stage',
    # sizes=[2],
    # capsize=.05,
    dodge=True,
    errwidth=linewidth
)
plt.setp(points.collections, sizes=[markersize])


ax1.set_ylim([0, None])
ax1.set_xlim([-0.5, 3.5])

ax1.set_ylabel('Cell area [$\mathrm{\mu m^2}$]')
ax1.set_ylim(0, 70)
ax1.set_yticks([0, 20, 40, 60])

ax1.set_xlabel('')
ax1.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

leg_handles = ax1.get_legend_handles_labels()[0]
ax1.legend(leg_handles, ['HC', 'SC'], title='')



points = sns.pointplot(
    data=cells.loc[cells['is_HC']],
    ax=ax2,
    x='stage',
    y='normalized_area_cells',
    hue='position',
    hue_order=['25I', '75I'],
    palette='Oranges',
    join=False,
    ci='sd',
    # capsize=.05,
    dodge=True,
    errwidth=linewidth
)
plt.setp(points.collections, sizes=[markersize])


leg_handles = ax2.get_legend_handles_labels()[0]
ax2.legend(leg_handles, ['25I', '75I'], title='')


ax2.hlines(y=[1], xmin=-1, xmax=5, ls=':', color='gray')

ax2.set_ylim([0, 5])
ax2.set_xlim([-0.5, 3.5])

ax2.set_ylabel('Normalised HC area')

ax2.set_xlabel('')
ax2.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

plt.savefig(f"{figure_path}/area_development__explants.svg",format='svg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])
fig.tight_layout()
# plt.gcf().subplots_adjust(bottom=0.15)

sns.pointplot(
    data=tissue_data.loc[tissue_data['position']=='25I'],
    x='stage',
    y='SC_area__no_norm',
    # palette='Oranges',
    color='gray',
    join=False,
    errorbar=np.var,
    capsize=.05,
    dodge=True,
    # scale=0.8,
    # errwidth=0.6
)

sns.pointplot(
    data=tissue_data.loc[tissue_data['position']=='25I'],
    x='stage',
    y='HC_area__no_norm',
    # palette='Oranges',
    color='red',
    join=False,
    errorbar='sd',
    capsize=.05,
    dodge=True,
    # scale=0.8,
    # errwidth=0.6
)

# sns.lineplot(data=cells.loc[cells['is_HC']],
#              x='stage',
#              y='normalized_area_cells',
#              hue='position', hue_order=['25I', '75I'],
#              palette='winter',
#              dashes=False,
#              ci='sd',
#              err_style='bars',
#              zorder=0,
#             )

ax.set_ylim([0, None])
ax.set_xlim([-0.5, 3.5])
ax.set_xticklabels([8, 10, 12, 14])

ax.set_ylabel('Cell area [$\mu m^2 $]')
ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

# leg_handles = ax.get_legend_handles_labels()[0]
# ax.legend(leg_handles, ['SC', 'HC'], title='Cell type')


plt.savefig(f"{figure_path}/HC_area__no_norm__tissues.svg",format='svg')

In [ ]:
def x_to_xtick(x, *, x_ticks):
    for i in range(len(x_ticks)):
        if x == x_ticks[i]:
            return 2 * i + 1
    raise

def histplot_violin(
    hist, *, 
    data_column, ax, label, 
    x_ticks, x_tick_labels, y_ticks, y_tick_labels, 
    set_colors, set_background=[],
    means=None,
    hist_split=None,
    means_split=None,
    split_color='green'
):
    ax.set_xlim([0, 2*len(x_ticks)])
    ax.set_xticks(np.arange(1, 2*len(x_ticks)+1, 2), x_tick_labels)
    ax.tick_params(axis='x', which='major', pad=0, rotation = 30)
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(2))
    for label in ax.get_xmajorticklabels():
        label.set_horizontalalignment("right")
    
    ax.set_yticks(
        np.arange(
            y_ticks[0],
            y_ticks[-1]+1,
            round((y_ticks[-1])/(len(y_tick_labels)-1))
        ),
        y_tick_labels
        )
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1))
    # ax.set_aspect(0.8)

    boxes = []
    colors = []
    for i, color in enumerate(set_background):
        if len(color) == 0:
            continue

        boxes.append(
            Rectangle(
                (2*i, 0),
                width=2,
                height=len(y_ticks),
                ls='',
                lw=None,
            )
        )
        colors.append(color)
    ax.add_collection(
        PatchCollection(
            boxes,
            facecolor=colors,
            edgecolor='None',
            alpha=0.2,
        )
    )

    boxes = []
    colors = []
    for i, (xtick, color) in enumerate(zip(x_ticks, set_colors)):
        _d = hist.loc[hist[data_column] == xtick]
        if hist_split is not None:
            split_d = hist_split.loc[hist_split[data_column] == xtick]

        for N in y_ticks:
            locx = x_to_xtick(xtick, x_ticks=x_ticks)
            count = _d[N].iloc[0]
            if np.isnan(count):
                count = 0

            boxes.append(
                Rectangle(
                    (locx, N - 0.5),
                    width=-count,
                    height=1,
                    ls='',
                    lw=0,
                )
            )
            colors.append(color)
            
            if hist_split is None:
                boxes.append(
                    Rectangle(
                        (locx, N - 0.5),
                        width=count,
                        height=1,
                        ls='',
                        lw=0,
                    )
                )
                colors.append(color)
            else:
                count = split_d[N].iloc[0]
                boxes.append(
                    Rectangle(
                        (locx, N - 0.5),
                        width=count,
                        height=1,
                        ls='',
                        lw=0,
                    )
                )
                colors.append(split_color)
        
        # plot mean as hline
        if means is not None and len(means):
            xtick = x_ticks[i]
            mean = means[i]
            count = _d[round(mean)].iloc[0]
            if means_split is None:
                xmax = 2*i + 1 + count
            else:
                xmax = 2*i + 1

            ax.hlines(
                y=mean,
                xmin=2*i + 1 - count,
                xmax=xmax,
                color='black',
                linestyles='dashed',
                linewidths=linewidth
            )

        if means_split is not None and len(means_split):
            xtick = x_ticks[i]
            mean = means_split[i]
            count = split_d[round(mean)].iloc[0]
            ax.hlines(
                y=mean,
                xmin=2*i + 1,
                xmax=2*i + 1 + count,
                color='black',
                linestyles='dashed',
                linewidths=linewidth
            )


    pc = PatchCollection(boxes, facecolor=colors)
    histogram = ax.add_collection(pc)
    # legend_elements = [
    #     mpl.patches.Patch(label='stage: {}'.format(stage)),
    #     # mpl.patches.Patch(facecolor='orange', edgecolor='orange', label='Theory')
    # ]
    # if label is not None:
    #     print(label)
    #     if type(label) is not str:
    #         label = 'E14'
    #     # NOTE weird error
    #     ax.set_title('{:s}'.format(str(label)), x=0.85, y=0.8, fontsize='small')
    # ax.legend(title='stage: {}'.format(stage), loc='upper right')
    # ax.legend(handles=legend_elements, loc='upper right')
    ax.set_ylabel('Neighbour numbers')
    ax.set_ylim(0, len(y_ticks)-1)


def histplot_violin_col(*, hist, col_labels, col_axes, col_column, means=None, means_split=None, **kwargs):
    hist = hist.reset_index()
    if means_split is None:
        means_split = []
        for a in col_axes:
            means_split.append(None)
    if means is None:
        means = []
        for a in col_axes:
            means.append(None)
    for (col, ax, _means, _means_split) in zip(col_labels, col_axes, means, means_split):
        histplot_violin(
            hist.loc[hist[col_column] == col], 
            ax=ax,
            label=col, 
            means=_means, means_split=_means_split,
            **kwargs
        )


In [ ]:

for position in ['25I', '50I', '75I']:
    figsize=[6.7, 1.9]
    fig, axes = plt.subplots(1, 4, figsize=figsize, sharex=True, sharey=True)
    (ax1, ax2, ax3, ax4) = axes

    for ax in axes:
        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)

    stages = [
        'E8',
        'E10',
        'E12',
        'E14',
    ]

    y_min = 0
    y_max=12
    y_ticks = np.arange(0, y_max+1)
    y_tick_labels = [
        '0',# '',
        '3',# '',
        '6',# '',
        '9',# '',
        '12',
    ]

    data = cells.loc[cells['position'] == position]

    hc = data.loc[data['is_HC']]
    sc = data.loc[~data['is_HC']]


    # HC neighborhood
    hc_norm = hc.groupby(by='stage')['num_neighbors'].count()

    _hist = [hc.loc[hc['num_neighbors'] == i].groupby(by='stage')['num_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'H'

    hist = _hist

    ### HH
    _hist = [hc.loc[hc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'HH'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    ### HS
    hc['num_support_neighbors'] = hc['num_neighbors'] - hc['num_hair_neighbors']
    _hist = [hc.loc[hc['num_support_neighbors'] == i].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(hc_norm, axis=0)
    _hist['type'] = 'HS'

    hist = pd.concat([hist, _hist], axis=0, join='outer')


    # SC neighborhood
    sc_norm = sc.groupby(by='stage')['num_neighbors'].count()

    _hist = [sc.loc[sc['num_neighbors'] == i].groupby(by='stage')['num_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'S'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    # SH
    _hist = [sc.loc[sc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'SH'

    hist = pd.concat([hist, _hist], axis=0, join='outer')

    # SS
    sc['num_support_neighbors'] = sc['num_neighbors'] - sc['num_hair_neighbors']
    _hist = [sc.loc[sc['num_support_neighbors'] == i].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
    _hist = pd.concat(_hist, axis=1, keys=y_ticks)
    _hist = _hist.divide(sc_norm, axis=0)
    _hist['type'] = 'SS'


    # concat
    hist = pd.concat([hist, _hist], axis=0, join='outer')
    hist = hist.reset_index()
    hist = hist.set_index(['stage', 'type'])

    hist = hist.reset_index()


    fig, axes = plt.subplots(1, 4, figsize=figsize, sharex=True, sharey=True)
    (ax1, ax2, ax3, ax4) = axes

    for ax in axes:
        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)

    x_ticks = [
        'H',
        'HH',
        # 'HS',
        'S',
        'SH',
        'SS'
    ]
    x_tick_labels = [
        'HC (all)',
        'HC-HC',
        #'', 'HC-SC',
        'SC (all)', 
        'SC-HC',
        'SC-SC',
    ]


    set_colors = [
        'red',
        'orangered',
        # 'gold',
        'dimgrey',
        'darkgray',
        'silver'
    ]

    set_background = [
        # 'coral',
        # '',
        # 'gray',
    ]

    means = []

    for stage in ['E8', 'E10', 'E12', 'E14']:
        mean_data = data.loc[data['stage'] == stage]
        tmp = [
            mean_data.loc[mean_data['is_HC']]['num_neighbors'].mean(),
            mean_data.loc[mean_data['is_HC']]['num_hair_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_hair_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_neighbors'].mean() - mean_data.loc[~mean_data['is_HC']]['num_hair_neighbors'].mean(),
        ]
        means.append(tmp)

    histplot_violin_col(
        hist=hist,
        col_column='stage',
        col_labels=stages,
        col_axes=[ax1, ax2, ax3, ax4],
        x_ticks=x_ticks,
        x_tick_labels=x_tick_labels,
        y_ticks=y_ticks,
        y_tick_labels=y_tick_labels,
        set_colors=set_colors,
        data_column='type',
        set_background=set_background,
        means=means,
    )


    fig.tight_layout()

    plt.savefig(f"{figure_path}/neighborhood__{position}.svg")


In [ ]:
figsizes=dict({
    'Vangl2': [1.6, 1.475],
    'MLCK': [1.3, 1.3],
    'ROCK': [1.4, 1.2],
    'ROCK_16hrs': [1.4, 1.2],
    'MLCK_16hrs': [1.4, 1.2],
    'PTX': [1.6, 1.4]
})
for Series in cells_experiments['Experiment_Series'].unique():
    if Series == 'Control':
        continue
    for num_neighbors, description in zip(['num_neighbors', 'num_neighbors__corr'], ['', '__corrected']):
        if Series in figsizes:
            figsize=figsizes[Series]
        else:
            figsize=[2., 2.]
        fig, ax = plt.subplots(1, 1, figsize=figsize, sharex=True, sharey=True)

        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1./4)
        for axis in ['top','right']:
            ax.spines[axis].set_linewidth(0)
        ax.tick_params(which='both', width=1./4)

        stages = [
        'E10',
        # 'E10',
        # 'E10',
        # 'E10',
        ]
        position = '25I'

        y_min = 0
        y_max=10
        y_ticks = np.arange(0, y_max+1)
        y_tick_labels = [
        '0',# '',
        '3',# '',
        '6',# '',
        '9',# '',
        # '12',
        ]


        histograms = dict()
        means = dict()
        for is_Control in [True, False]:
            data = cells_experiments.loc[
                # (cells_experiments['position'] == position) &
                # (cells_experiments['stage'] == stage) &
                (cells_experiments['Experiment_Series'] == Series) &
                (cells_experiments['is_Control'] == is_Control)
            ]

            hc = data.loc[data['is_HC']].copy()
            sc = data.loc[~data['is_HC']].copy()


            # HC neighborhood
            hc_norm = hc.groupby(by='stage')[num_neighbors].count()

            _hist = [hc.loc[(hc[num_neighbors] > i - 0.5) & (hc[num_neighbors] < i + 0.5)].groupby(by='stage')[num_neighbors].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'H'

            hist = _hist

            ### HH
            _hist = [hc.loc[hc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'HH'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            ### HS
            hc['num_support_neighbors'] = hc[num_neighbors] - hc['num_hair_neighbors']
            _hist = [hc.loc[(hc['num_support_neighbors'] > i - 0.5) & (hc['num_support_neighbors'] < i + 0.5)].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(hc_norm, axis=0)
            _hist['type'] = 'HS'

            hist = pd.concat([hist, _hist], axis=0, join='outer')


            # SC neighborhood
            sc_norm = sc.groupby(by='stage')[num_neighbors].count()

            _hist = [sc.loc[(sc[num_neighbors] > i - 0.5) & (sc[num_neighbors] < i + 0.5)].groupby(by='stage')[num_neighbors].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'S'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            # SH
            _hist = [sc.loc[sc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'SH'

            hist = pd.concat([hist, _hist], axis=0, join='outer')

            # SS
            sc['num_support_neighbors'] = sc[num_neighbors] - sc['num_hair_neighbors']
            _hist = [sc.loc[(sc['num_support_neighbors'] > i - 0.5) & (sc['num_support_neighbors'] < i + 0.5)].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
            _hist = pd.concat(_hist, axis=1, keys=y_ticks)
            _hist = _hist.divide(sc_norm, axis=0)
            _hist['type'] = 'SS'


            # concat
            hist = pd.concat([hist, _hist], axis=0, join='outer')
            hist = hist.reset_index()
            hist = hist.set_index(['stage', 'type'])

            hist = hist.reset_index()
            histograms[is_Control] = hist

            means[is_Control] = [[
                hc[num_neighbors].mean(),
                hc['num_hair_neighbors'].mean(),
                sc[num_neighbors].mean(),
                sc['num_hair_neighbors'].mean(),
                sc[num_neighbors].mean() - sc['num_hair_neighbors'].mean(),
            ]]

        x_ticks = [
        'H',
        'HH',
        # 'HS',
        'S',
        'SH',
        'SS'
        ]
        x_tick_labels = [
        'HC (all)',
        'HC-HC',
        #'', 'HC-SC',
        'SC (all)', 
        'SC-HC',
        'SC-SC',
        ]


        set_colors = [
        'red',
        'orangered',
        # 'gold',
        'dimgrey',
        'darkgray',
        'silver'
        ]

        set_background = [
        # 'coral',
        # '',
        # 'gray',
        ]

        histplot_violin_col(
            hist=histograms[True],
            col_column='stage',
            col_labels=stages,
            col_axes=[ax],
            x_ticks=x_ticks,
            x_tick_labels=x_tick_labels,
            y_ticks=y_ticks,
            y_tick_labels=y_tick_labels,
            set_colors=set_colors,
            data_column='type',
            set_background=set_background,
            hist_split=histograms[False],
            means=means[True],
            means_split=means[False],
            split_color='blue',
        )


        plt.savefig(f"{figure_path}/neighborhood__{Series}{description}.svg")
        plt.close()

In [ ]:
fig, ax = plt.subplots(1, 1)

data = cells_experiments.loc[
    (
        (cells_experiments['Experiment_Series']=='MLCK') | 
        (cells_experiments['Experiment_Series']=='MLCK_16hrs')
    ) &
    (cells_experiments['is_HC'])
]

sns.violinplot(
    data=data,
    x='Experiment_Series',
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
    # hue='is_Control',
    # hue_order=[True, False],
)

ax.set_ylim([0, 3])


box_pairs=[
    (('MLCK', True), ('MLCK', False)),
    (('MLCK', True), ('MLCK_16hrs', True)),
    (('MLCK', False), ('MLCK_16hrs', False)),
    (('MLCK_16hrs', True), ('MLCK_16hrs', False))
    # ((True, 'Vangl2Control'), (True, 'Vangl2KO')),
    # ((False, 'Vangl2Control'), (False, 'Vangl2KO')),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=data,
    x='Experiment_Series',
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()

In [ ]:
fig, ax = plt.subplots(1, 1)

data = cells_experiments.loc[
    (
        (cells_experiments['Experiment_Series']=='ROCK') | 
        (cells_experiments['Experiment_Series']=='ROCK_16hrs')
    ) &
    (cells_experiments['is_HC'])
]

sns.violinplot(
    data=data,
    x='Experiment_Series',
    order=['ROCK', 'ROCK_16hrs'],
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
    # hue='is_Control',
    # hue_order=[True, False],
)

ax.set_ylim([0, 3])


box_pairs=[
    (('ROCK', True), ('ROCK', False)),
    (('ROCK', True), ('ROCK_16hrs', True)),
    (('ROCK', False), ('ROCK_16hrs', False)),
    (('ROCK_16hrs', True), ('ROCK_16hrs', False))
    # ((True, 'Vangl2Control'), (True, 'Vangl2KO')),
    # ((False, 'Vangl2Control'), (False, 'Vangl2KO')),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=data,
    x='Experiment_Series',
    order=['ROCK', 'ROCK_16hrs'],
    hue='is_Control', hue_order=[True, False],
    split=True,
    y='num_hair_neighbors',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()

In [ ]:
((tissue_data['stage'] == 'E12') * (tissue_data['position'] == '25I'))

In [ ]:
# print(tissue_data.loc[(tissue_data['position'] == '25I')].groupby(by='stage').mean()['HC_area'])
# print(tissue_data.loc[(tissue_data['position'] == '25I')][['stage', 'HC_area']])
# print(tissue_data.loc[(tissue_data['position'] == '25I') * (tissue_data['HC_area'] )][['stage', 'HC_area']])

In [ ]:
HC_averages = cells.loc[cells['is_HC'] * cells['position'] == '25I'].groupby(by=['file_id', 'stage']).mean()['normalized_area_cells']
print(HC_averages)
HC_averages = HC_averages.reset_index()
print(HC_averages[['stage', 'normalized_area_cells']].groupby(by='stage').apply(np.percentile, q=50, method='nearest'))

In [ ]:
for stage in ['E8', 'E10', 'E12', 'E14']:
    tmp = tissue_data.loc[(tissue_data['position'] == '25I') * (tissue_data['stage'] == stage)]
    tmp['zscore__HC_area'] = np.abs(scipy.stats.zscore(tmp['HC_area']))
    tmp = tmp.sort_values(by='zscore__HC_area')
    print(stage, tmp['HC_area'].mean(), tmp['HC_area'].std())
    print(tmp[['HC_area', 'zscore__HC_area', 'file_id']])
    print(tmp.loc[tmp['zscore__HC_area'] < tmp['zscore__HC_area'].median()][['HC_area', 'zscore__HC_area', 'filename', 'file_id']])
    print()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/4, 5.9/3])

mask = np.where(
    cells['HC_normalized_area']
)

sns.histplot(
    data=cells.loc[cells['stage'] == 'E10'],
    x='num_neighbors',
    binrange=[3, 12],
    discrete=True,
    hue='is_HC',
    palette=['gray', 'red'],
    multiple='stack',
    stat='probability',
    common_norm=True,
    legend=False,
    alpha=0.5
)

__d = cells.loc[cells['stage'] == 'E10']
__hc = __d.loc[__d['is_HC']]
ax.vlines(
    x=[
        __d['num_neighbors'].mean(),
        __d.loc[__d['is_HC'], 'num_neighbors'].mean(),
        __d.loc[~__d['is_HC'], 'num_neighbors'].mean(),
    ],
    ymin=[
        0,
        0,
        __hc.loc[__hc['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
    ],
    ymax=[
        __d.loc[__d['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
        __hc.loc[__hc['num_neighbors']==5, 'num_neighbors'].count() / __d['num_neighbors'].count(),
        __d.loc[__d['num_neighbors']== 6, 'num_neighbors'].count() / __d['num_neighbors'].count(),
    ],
    colors=['black', 'red', 'gray'],
    lw=0.5,
    ls='dashed',
)

ax.set_xlim([2.5, 10.5])
ax.set_xticks([4, 6, 8, 10])
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

# ax.set_ylim([0, 3050])
ax.set_ylim([0, 0.4])

ax.set_xlabel('Neighbour number')
ax.set_ylabel('Frequency')


plt.savefig(f"{figure_path}/neighbours_hc_vs_sc.svg")

In [ ]:
def plot_hist(*, data, x, discrete_bins=False,  **kwargs):
    ax = plt.gca()

    sns.histplot(
        data=data,
        x=x,
        hue='is_HC',
        palette=['gray', 'red'],
        multiple='stack',
        stat='probability',
        # common_norm=True,
        legend=False,
        alpha=0.5,
        **kwargs
    )

    __hc = data.loc[data['is_HC']]
    hc_mean = data.loc[data['is_HC'], x].mean()
    sc_mean = data.loc[~data['is_HC'], x].mean()

    vlines_kwargs = dict({
        'lw': linewidth,
        'ls': 'dashed',
        'colors': ['black', 'red', 'gray'],
    })

    if discrete_bins:
        __mean = round(data[x].mean())
        __hc_mean = round(hc_mean)
        __sc_mean = round(sc_mean)

        plt.vlines(
            x=[
                data[x].mean(),
                hc_mean,
                sc_mean,
            ],
            ymin=[
                0,
                0,
                __hc.loc[np.round(__hc[x])==np.round(__sc_mean), x].count() / data[x].count(),
            ],
            ymax=[
                data.loc[np.round(data[x])==np.round(__mean), x].count() / data[x].count(),
                __hc.loc[np.round(__hc[x])==np.round(__hc_mean), x].count() / data[x].count(),
                data.loc[np.round(data[x])==np.round(__sc_mean), x].count() / data[x].count(),
            ],
            **vlines_kwargs
        )
    else:
        ymax = ax.get_ylim()[1]
        plt.vlines(
            x=[
                data[x].mean(),
                hc_mean,
                sc_mean,
            ],
            ymin=[
                0,
                0,
                0.,
            ],
            ymax=[
                ymax,
                ymax,
                ymax
            ],
            **vlines_kwargs
        )

        

    legend = ax.legend(
        labels=['all','HC', 'SC'],
    )

In [ ]:
print(cells.loc[(cells['position']=='25I') & (cells['stage']=='E10')].groupby(by='is_HC')['normalized_area_cells'].mean())

In [ ]:
treatments = [
    'Control',

    'MLCKdmso-4hrs',
    'MLCKdmso-16hrs',
    'MLCKml7-4hrs',
    'MLCKml7-16hrs',
    
    'Vangl2Control',
    'Vangl2KO',

    'Atoh1control',
    'Atoh1KO',

    'ROCKcontrol-4hrs',
    'ROCKcontrol-16hrs',
    'ROCKy27632-4hrs',
    'ROCKy27632-16hrs',
    
    'PTXcontrol',
    'PTXinhibition',
]
descriptions = [
    'Fixed BP (Control, 25I @ E10)',

    'MLCK (DMSO, 4hrs)',
    'MLCK (DMSO, 16hrs)',
    'MLCK (4hrs)',
    'MLCK (washoff, 16hrs)',

    'Vangl2 (Control)',
    'Vangl2-KO',

    'Atoh1 (Control)',
    'Atoh1-KO',

    'ROCK (DMSO, 4hrs)',
    'ROCK (DMSO, 16hrs)',
    'ROCK (4hrs)',
    'ROCK (washoff, 16hrs)',

    'PTX (DMSO)',
    'PTX (4hrs)',
]
for treatment, description in zip(treatments, descriptions):
    data = cells_experiments.loc[cells_experiments['Treatment'] == treatment]
    data = data.loc[data['stage']=='E10']
    data = data.loc[data['position']=='25I']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[2*5.9/3,5.9/4*3/2])

    plt.sca(ax1)
    plot_hist(data=data, x='normalized_area_cells', discrete = False)
    ax1.set_xlim(0, 3)


    plt.sca(ax2)
    plot_hist(data=data, x=f'num_neighbors', discrete = True, discrete_bins=True)
    ax2.set_xlim(3, 10)

    ax2.set_xlim([3, 10])
    ax2.set_xticks([4, 6, 8, 10])
    ax2.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax2.set_xlabel('Neighbour number')

    plt.suptitle(f'{description}')

    plt.savefig(f"{figure_path}/neighbours_hc_vs_sc__{treatment}.svg")
    plt.close()


    fig, ax = plt.subplots(1, 1, figsize=[5.9/4,5.9/4*3/2])


    plot_hist(data=data, x=f'num_neighbors__corr', discrete=False, discrete_bins=True, bins=np.arange(2.5, 10.5, 1))
    ax.set_xlim(3, 10)

    ax.set_xlim([3, 10])
    ax.set_xticks([4, 6, 8, 10])
    # ax.set_xticks([3, 6, 9])
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    ax.set_xlabel('Neighbour number')

    ax.set_ylim([0, 0.4])
    ax.set_yticks([0., 0.2, 0.4])

    # plt.suptitle(f'{description}')
    plt.savefig(f"{figure_path}/neighbours_hc_vs_sc__{treatment}__corrected.svg")
    plt.close()
    # break

In [ ]:
for treatment in cells_experiments['Experiment_Series'].unique():
    if treatment == 'Control':
        continue

    print(treatment)


    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[4, 2])
    fig.tight_layout()

    data = cells_experiments.loc[cells_experiments['Experiment_Series']==treatment].copy()
    data.loc[data['is_HC'], 'neighbourhood'] ='HC'
    data.loc[~data['is_HC'], 'neighbourhood'] ='SC'

    kwargs = dict({
        'order': ['HC', 'SC'],
        'x': 'neighbourhood',
        'hue': 'is_Control',
        'hue_order': [True, False],
        'split': True,
        'inner': 'quartiles',
    })

    sns.violinplot(
        data=data,
        y=f'num_neighbors__corr',
        ax=ax1,
        **kwargs
    ) 
    
    sns.violinplot(
        data=data,
        y='num_hair_neighbors',
        ax=ax2,
        **kwargs
    )

    box_pairs=[
        (('HC', True), ('HC', False)),
        (('SC', True), ('SC', False)),
    ]

    annotator = Annotator(
        ax1,
        box_pairs,
        data=data,
        x='neighbourhood',
        order=['HC', 'SC'],
        y=f'num_neighbors__corr',
        hue='is_Control',
        split=True,
    )
    annotator.configure(
        test='Mann-Whitney',
        line_width=0.75,
    )
    annotator.apply_and_annotate()

    annotator = Annotator(
        ax2,
        box_pairs,
        data=data,
        x='neighbourhood',
        order=['HC', 'SC'],
        y='num_hair_neighbors',
        hue='is_Control',
        split=True,
    )
    annotator.configure(
        test='Mann-Whitney',
        line_width=0.75,
    )
    annotator.apply_and_annotate()

    ax1.set_ylim([3, 10])
    ax1.set_xlabel("")
    ax1.set_ylabel("Cell contacts")

    ax2.set_ylim([0, 5])
    ax2.set_xlabel("")
    ax2.set_ylabel("HC contacts")

    ax1.get_legend().remove()
    # ax2.legend(loc='upper left')

    plt.savefig(f"{figure_path}/neighborhood__{treatment}__significance.svg",format='svg')
    plt.close()

In [ ]:
print(cells_experiments.loc[cells_experiments['is_HC']].groupby(by='Treatment')['num_hair_neighbors'].apply(lambda data: [data.loc[data > 0.01].count() , data.count()]))

In [ ]:
fig, (ax1, ax2 )= plt.subplots(2, 1, figsize=[10, 10])

mask = np.where(
    cells_experiments['Treatment'] == 'MLCKdmso-4hrs',
    True,
    cells_experiments['Treatment'] == 'MLCKml7-4hrs',
)
mask = np.where(
    cells_experiments['Treatment'] == 'MLCKdmso-16hrs',
    True,
    mask
)
mask = np.where(
    cells_experiments['Treatment'] == 'MLCKml7-16hrs',
    True,
    mask
)
mask = np.where(
    cells_experiments['Treatment'] == 'Control',
    True,
    mask
)
mask = np.where(
    cells_experiments['Treatment'] == 'Vangl2Control',
    True,
    mask
)
mask = np.where(
    cells_experiments['Treatment'] == 'Vangl2KO',
    True,
    mask
)
data = cells_experiments.loc[mask]

plt.sca(ax1)
sns.violinplot(
    data=data.loc[data['is_HC']],
    x='Treatment',
    y='num_neighbors__corr',
    order=['Vangl2KO', 'Vangl2Control', 'Control', 'MLCKdmso-4hrs', 'MLCKml7-4hrs', 'MLCKdmso-16hrs', 'MLCKml7-16hrs'],
    inner='quartile',
    scale='width',
    bw=0.1,
)


box_pairs=[
    ('Control', 'MLCKdmso-4hrs'),
    ('Control', 'MLCKdmso-16hrs'),
    ('Control', 'MLCKml7-16hrs'),
    ('MLCKdmso-4hrs', 'MLCKml7-4hrs'),
    ('MLCKdmso-16hrs', 'MLCKml7-16hrs'),
    ('MLCKdmso-4hrs', 'MLCKdmso-16hrs'),
    ('MLCKdmso-4hrs', 'MLCKml7-16hrs'),
    ('MLCKml7-4hrs', 'MLCKml7-16hrs'),
    ('MLCKml7-4hrs', 'MLCKdmso-16hrs'),
    ('Control', 'Vangl2Control'),
    ('Vangl2Control', 'Vangl2KO'),
    ('Vangl2KO', 'MLCKml7-4hrs'),
]

annotator = Annotator(
    ax1,
    box_pairs,
    data=data.loc[data['is_HC']],
    x='Treatment',
    y='num_neighbors__corr',
    order=['Vangl2KO', 'Vangl2Control', 'Control', 'MLCKdmso-4hrs', 'MLCKml7-4hrs', 'MLCKdmso-16hrs', 'MLCKml7-16hrs'],
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()


plt.sca(ax2)

sns.violinplot(
    data=data.loc[~data['is_HC']],
    x='Treatment',
    y='num_neighbors__corr',
    order=['Vangl2KO', 'Vangl2Control', 'Control', 'MLCKdmso-4hrs', 'MLCKml7-4hrs', 'MLCKdmso-16hrs', 'MLCKml7-16hrs'],
    inner='quartile',
    scale='width',
    bw=0.1,
)

annotator = Annotator(
    ax2,
    box_pairs,
    data=data.loc[~data['is_HC']],
    x='Treatment',
    y='num_neighbors__corr',
    order=['Vangl2KO', 'Vangl2Control', 'Control', 'MLCKdmso-4hrs', 'MLCKml7-4hrs', 'MLCKdmso-16hrs', 'MLCKml7-16hrs'],
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()



ax1.set_ylim([3, 20])
ax2.set_ylim([3, 22])
ax1.set_title('HC')
ax2.set_title('SC')


# Area mapped

In [ ]:
def tissue_data_errorbar(
    *,
    x: str='HC_area',
    xlabel: str='Normalised HC area',
    y: str,
    ylabel: str=None,
    hue='Stage',
    palette=palette_stages,
    style='P-D position',
    plot_control: bool=True, plot_errorbar=True,
    plot_treatment: bool=False, plot_errorbar_treatment=True,
    xlim=[0., 4.], ylim=None, xticks=None, yticks=None,
    figsize=[5.9/3, 5.9/4],
    savefig: str=None,
    mask=None,
    mask_treatment=None,
    tight_layout=True,
):  
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    if tight_layout:
        fig.tight_layout()

    data = tissue_data.copy()
    if mask is not None:
        data = tissue_data.loc[mask]

    if plot_control:
        sns.scatterplot(
            data=data,
            x=x,
            y=y,
            hue=hue,
            palette=palette,
            style=style,
            # size='S-I position',
            # size_order=['Superior', 'Mid', 'Inferior']
        )
        if plot_errorbar:
            ax.errorbar(
                x=data[x],
                y=data[y],
                xerr=data["{}__std".format(x) ] if "{}__std".format(x) in data.columns else None,
                yerr=data["{}__std".format(y) ] if "{}__std".format(y) in data.columns else None,
                ls='',
                color='black', 
                linewidth=0.2, alpha=0.4,
                zorder=0
            )

    if plot_treatment:
        __data = __tissue_data.copy()
        if mask_treatment is not None:
            __data = __data.loc[mask_treatment]
        __data = __data.loc[__data['Experiment'] != 'Control']
        __data['__Experiment'] = __data['Experiment']
        __data['Experiment'] = __data['Experiment_Series']
        style_order=['Live', 'MLCK', 'MLCK_16hrs', 'Vangl2', 'ROCK', 'ROCK_16hrs', 'Atoh1', 'PTX']
        __data.Experiment = __data.Experiment.astype("category")
        __data.Experiment = __data.Experiment.cat.set_categories(style_order)
        sns.scatterplot(
            data=__data,
            x=x,
            y=y,
            hue='is_Control',
            hue_order=[True, False],
            palette=['blue', 'red'],
            style='Experiment',
        )
        if plot_errorbar_treatment:
            ax.errorbar(
                x=__data[x],
                y=__data[y],
                xerr=__data["{}__std".format(x) ] if "{}__std".format(x) in __data.columns else None,
                yerr=__data["{}__std".format(y) ] if "{}__std".format(y) in __data.columns else None,
                ls='',
                color='black', 
                linewidth=0.2, alpha=0.4,
                zorder=0
            )

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel if ylabel is not None else y)
    if xticks is not None:
        ax.set_xticks(xticks)
    if yticks is not None:
        ax.set_yticks(yticks)

    mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='xx-small')


    if savefig is not None:
        plt.savefig(f"{figure_path}/area_mapped__{savefig}.svg")


In [ ]:
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    savefig='HC_neighbours',
)

In [ ]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour no.',
    x='SC_area', xlabel='Normalised SC area',
    xlim=[0,1.5], ylim=[3, 8],
    savefig='SC_neighbours__SC_area',
)

In [ ]:

np.polyfit(tissue_data['HC_area'], tissue_data['subordination_number'], 1)

m_HC = np.polyfit(tissue_data['HC_area'], tissue_data['coordination_number'], 1)[0]
m_SC = np.polyfit(tissue_data['HC_area'], tissue_data['subordination_number'], 1)[0]
def correct_neighbors(nbs, area, is_HC):
    if is_HC:
        m = m_HC
    else:
        m = m_SC
    return nbs  + m * (1. - area)

tissue_data['coordination_number__corr'] = correct_neighbors(tissue_data['coordination_number'], tissue_data['HC_area'], True)
tissue_data['subordination_number__corr'] = correct_neighbors(tissue_data['subordination_number'], tissue_data['HC_area'], False)
tissue_data['delta_area'] = tissue_data['subordination_number__corr'] - tissue_data['coordination_number__corr']

In [ ]:
tissue_data_errorbar(
    y='coordination_number__corr', ylabel='HC neighbour no.',
    ylim=[3, 11],
)

In [ ]:
tissue_data_errorbar(
    y='delta_area', ylabel='HC neighbour no.',
    # ylim=[3, 11],
)

In [ ]:
tissue_data_errorbar(
    y='subordination_number__corr', ylabel='HC neighbour no.',
    ylim=[3, 11],
    savefig='total_neighbours',
)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=tissue_data, x='HC_area', y='coordination_number')
ax.plot(np.linspace(0, 4, 101), np.linspace(0, 4, 101) * 1.514 + 3.865)

In [ ]:
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    plot_treatment=True,
    savefig='HC_neighbours__w_treatment',
    # mask_treatment=mask_treatment
)

In [ ]:
mask_treatment = __tissue_data['Experiment'] == 'Vangl2KO'
mask_treatment = np.where(
    __tissue_data['Experiment'] == 'Vangl2Control',
    True,
    mask_treatment
)
print(np.unique(__tissue_data.loc[mask_treatment]['filename']))
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    plot_treatment=True,
    savefig='HC_neighbours__Vangl2KO',
    mask_treatment=mask_treatment
)

In [ ]:
mask_treatment = __tissue_data['Experiment'] == 'MLCKdmso-4hrs'
mask_treatment = np.where(
    __tissue_data['Experiment'] == 'MLCKdmso-16hrs',
    True,
    mask_treatment
)
mask_treatment = np.where(
    __tissue_data['Experiment'] == 'MLCKml7-4hrs',
    True,
    mask_treatment
)
mask_treatment = np.where(
    __tissue_data['Experiment'] == 'MLCKml7-16hrs',
    True,
    mask_treatment
)
print(np.unique(__tissue_data.loc[mask_treatment]['filename']))
tissue_data_errorbar(
    y='coordination_number', ylabel='HC neighbour no.',
    ylim=[3, 11],
    plot_treatment=True,
    savefig='HC_neighbours__MLCK',
    mask_treatment=mask_treatment
)

In [ ]:
print(cells_experiments['Experiment'].unique())
for c in cells_experiments.columns:
    print(c)

In [ ]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour number',
    ylim=[3, 11],
    figsize=[2.75, 5.9/3],
    savefig='SC_neighbourhood',
)

In [ ]:
tissue_data_errorbar(
    y='subordination_number', ylabel='SC neighbour no.',
    ylim=[3, 9],
    plot_treatment=True,
    savefig='SC_neighbourhood__w_treatment',
)

In [ ]:
tissue_data_errorbar(
    y='neighbor_number', ylabel='Neighbour no.',
    ylim=[4, 8],
    savefig='topology',
)

In [ ]:
tissue_data_errorbar(
    y='area__no_norm', ylabel='Cell area [$\mu m^2$]',
    ylim=[0, None],
    savefig='tissue_size',
)

In [ ]:
tissue_data_errorbar(
    y='hexatic_order', ylabel='Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    savefig='hexatic_order',
)

In [ ]:
tissue_data_errorbar(
    y='hexatic_order_corrected', ylabel='cor. Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    savefig='hexatic_order_corrected',
)

In [ ]:
tissue_data_errorbar(
    y='hexatic_order_corrected', ylabel='cor. Hexatic order [$\psi_6$]',
    ylim=[0, 1],
    plot_treatment=True,
    savefig='hexatic_order_corrected__w_treatment',
)

In [ ]:
tissue_data_errorbar(
    y='HC_density', ylabel='HC density',
    ylim=[0, 0.4],
    figsize=[2, 2*2/3], tight_layout=False,
    savefig='HC_density',
)

In [ ]:
tissue_data_errorbar(
    y='HC_density', ylabel='HC number density',
    ylim=[0, 0.4],
    plot_treatment=True,
    savefig='HC_density__w_treatment',
)

In [ ]:
tissue_data_errorbar(
    y='polarity__variance', ylabel='polarity variance',
    # ylim=[0, 0.4],
    savefig='polarity_variance',
)

In [ ]:
tissue_data_errorbar(
    y='polarity__variance', ylabel='polarity variance',
    ylim=[0, None],
    plot_treatment=True,
    savefig='polarity_variance__w_treatment',
)

In [ ]:
tissue_data_errorbar(
    x='hexatic_order', xlabel='Hexatic order [$\psi_6$]',
    y='polarity__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, None],
    figsize=[2, 1.5],
    # plot_treatment=True,
    savefig='polarity_variance__vs_hexatic_order__no_correction',
)

In [ ]:
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, 1.],
    figsize=[3, 1.6],
    # plot_treatment=True,
    savefig='polarity_variance__vs_hexatic_order',
)

In [ ]:
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__stddev_angles', ylabel='Variation of polarity\n[c.s.d. in '+r'$\degree$]',
    style=None,
    xlim=[0, 0.8],
    ylim=[0, 120],
    xticks=[0., 0.4, 0.8],
    yticks=[0, 30, 60, 90, 120],
    figsize=[2., 1.6],
    plot_errorbar=False,
    savefig='polarity_stddev__vs_hexatic_order',
)

In [ ]:
mask = (tissue_data['position'] == '25I') | (tissue_data['position'] == '75I')
tissue_data_errorbar(
    x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
    y='polarity_corrected__variance', ylabel='Polarity variance',
    xlim=[0, 1.],
    ylim=[0, 1.],
    figsize=[3, 1.8],
    mask=mask,
    savefig='polarity_variance__vs_hexatic_order__25I_75I',
)

In [ ]:
for treatment in __tissue_data['Experiment_Series'].unique():
    tissue_data_errorbar(
        x='hexatic_order_corrected', xlabel='Hexatic order [$\psi_6$]',
        y='polarity_corrected__stddev_angles', ylabel='Variation of polarity\n[c.s.d. in '+r'$\degree$]',
        style=None,
        xlim=[0, 1.],
        ylim=[0, 140],
        yticks=[0, 30, 60, 90, 120, 140],
        figsize=[3, 1.6],
        plot_errorbar=False,
        plot_treatment=True, plot_errorbar_treatment=False,
        mask_treatment=(__tissue_data['Experiment_Series']==treatment),
        savefig='polarity_stddev__vs_hexatic_order__{}'.format(treatment),
    )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[2.4, 1.6])

sns.regplot(
    data=tissue_data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    # x_ci='sd', ci=None,
    color='black', scatter=False,
    line_kws={'ls': 'dashed', 'lw': linewidth},
    label='BP explants'
    # size='S-I position',
    # size_order=['Superior', 'Mid', 'Inferior']
)

data = __tissue_data.copy()
data['__Experiment'] = data['Experiment']
data['Experiment'] = data['Experiment_Series']
data = data.loc[
    (data['Experiment']=='MLCK') |
    (data['Experiment']=='ROCK') |
    (data['Experiment']=='PTX')
]
sns.scatterplot(
    data=data,
    x='hexatic_order_corrected',
    y='polarity_corrected__stddev_angles',
    style='is_Control',
    style_order=[True, False],
    # palette=['blue', 'red'],
    hue='Experiment',
)

order_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'hexatic_order_corrected'].min()
polarity_limit = tissue_data.loc[
    (tissue_data['stage']=='E12') | (tissue_data['stage']=='E14'), 
    'polarity_corrected__stddev_angles'].max()
ax.vlines(
    x=[order_limit],
    ymin=0, ymax=140,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)
ax.hlines(
    y=[polarity_limit],
    xmin=0, xmax=1,
    color='gray', alpha=0.7,
    lw=linewidth,
    zorder=-np.inf
)

# Create a Rectangle patch
rect = mpl.patches.Rectangle(
    (0, polarity_limit), 
    order_limit, 150,
    linewidth=0., edgecolor='none', 
    facecolor='red', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)
rect = mpl.patches.Rectangle(
    (order_limit, 0), 
    1., polarity_limit,
    linewidth=0., edgecolor='none', 
    facecolor='blue', alpha=0.1, 
    zorder=-np.inf
)
ax.add_patch(rect)

ax.set_xlim([0., 0.8])
ax.set_ylim([0., 140])
ax.set_yticks(range(0, 140, 30))

ax.set_xlabel('Hexatic order [$\psi_6$]')
ax.set_ylabel('Variation of polarity\n[c.s.d. in '+r'$\degree$]')

mpl.pyplot.legend(bbox_to_anchor=(1,1.025), fontsize='x-small')

plt.savefig(f"{figure_path}/area_mapped__polarity_stddev_vs_hexatic_order__inhibitions.svg")

In [ ]:
tissue_data_errorbar(
    y='num_next_HC_neighbors', ylabel='no. next HC neighbours',
    ylim=[0, None],
    plot_treatment=True,
    # savefig='polarity_variance__w_treatment',
)

# Stage evaluation

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5.9/5, 5.9/3])

points = sns.pointplot(
    data=cells,
    x='stage',
    y='hexatic_order_corrected',
    hue='position', hue_order=['25I', '75I'],
    # palette='Oranges',
    join=False,
    ci='sd',
    # capsize=.05,
    dodge=True,
    s=2,
    errwidth=linewidth,
)
plt.setp(points.collections, sizes=[markersize])

results__hexatic_order = pd.DataFrame()
results__hexatic_order['hexatic_order'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].mean()
results__hexatic_order['hexatic_order__std'] = cells.groupby(by=['stage', 'position'])['hexatic_order_corrected'].std()

print(results__hexatic_order.to_string())

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['25I', '75I'], title='')

ax.set_ylim([0, 1])

ax.set_xlabel('')
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'], fontsize=labelsize)

ax.set_ylabel('Hexatic order $\psi_6$')
ax.set_yticks([0., 0.5, 1.])


plt.savefig(f"{figure_path}/hexatic_vs_stage__explants.svg")

In [ ]:
fig, ax = plt.subplots(1, 1)


sns.pointplot(
    data=tissue_data,
    x='Stage',
    y='polarity_corrected__variance',
    hue='position', hue_order=['25I', '75I'],
    # palette='Oranges',
    join=False,
    ci='sd',
    err_style='bars',
    capsize=.05,
    dodge=True,
)

# ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

ax.set_ylabel('Circular variance of polarity (corr.)')



plt.legend(prop={'size': 'small'})

plt.savefig(f"{figure_path}/polarity_variance_vs_stage__explants.svg")

### Violin plots

In [ ]:
# fig, ax = plt.subplots(1, 1)

# x_ticks = ['', 'E8', '', 'E10', '', 'E12', '', 'E14', '']
# def x_to_locx(s: str):
#     if s == 'E8':
#         return 1
#     if s == 'E10':
#         return 3
#     if s == 'E12':
#         return 5
#     if s == 'E14':
#         return 7

# y_min = 3
# y_ticks = np.arange(3, 13)

# ax.set_xticks(np.arange(len(x_ticks)), x_ticks)
# ax.set_yticks(np.arange(len(y_ticks)), y_ticks)

# data = HC.loc[HC['position'] == '25S']

# norm = data.groupby(by='stage')['num_neighbors'].count()

# hist = [data.loc[data['num_neighbors'] == i].groupby(by='stage')['num_neighbors'].count() for i in y_ticks]
# hist = pd.concat(hist, axis=1, keys=y_ticks)
# hist = hist.divide(norm, axis=0)

# hist = hist.reset_index()
# hist.stage = hist.stage.astype("category")
# hist.stage.cat.set_categories(stage_ordering, inplace=True)
# hist = hist.sort_values(by='stage')

# boxes = []
# for stage in hist['stage']:
#     d = hist.loc[hist['stage'] == stage]
#     for N in y_ticks:
#         boxes.append(
#             Rectangle((x_to_locx(stage), N - y_min - 0.5),
#                       width=-d[N], height=1)
#             )

# pc = PatchCollection(boxes, color='blue')
# histogram = ax.add_collection(pc)
# legend_elements = [
#     mpl.patches.Patch(facecolor='blue', edgecolor='blue', label='Experiment'),
#     mpl.patches.Patch(facecolor='orange', edgecolor='orange', label='Theory')
# ]
# ax.legend(handles=legend_elements, loc='upper left')


In [ ]:
# fig, ax = plt.subplots(1, 1)

# data = cells.loc[cells['Experiment'] == 'control']
# data = data.loc[data['stage'] == 'E14']

# groubby_n = data.groupby(by=['Experiment', 'stage', 'position', 'sample_id', 'frame_nb', 'num_neighbors'])
# # groubby_n.apply(lambda d: print(d[['frame_nb', 'num_neighbors', 'normalized_area_cells']]))
# A_n = groubby_n.apply(lambda grp: grp['normalized_area_cells'].mean())
# A_n = A_n.rename('A_n').reset_index().sort_values(by=['Experiment', 'stage', 'position', 'sample_id'], )
# print(A_n)

# sns.scatterplot(
#     data=A_n,
#     x='num_neighbors',
#     y='A_n',
#     hue='stage',
#     hue_order=stage_ordering,
#     style='position',
#     style_order=position_ordering,
#     # size='sample_id',
#     palette=palette_stages,
#     # markers=True,
#     # ls='',
#     # dashes=False,
#     # ci='sd',
#     # err_style='bars'
# )
# sns.regplot(
#     data=A_n,
#     x='num_neighbors',
#     y='A_n',
#     scatter=False
# )
# sns.regplot(
#     data=A_n,
#     x='num_neighbors',
#     y='A_n',
#     scatter=False,
#     order=2
# )

# ax.legend(bbox_to_anchor=(1.01, 1.05))
# ax.set_title("all cells")

# _ns = np.array([i for i in range(2, 14)])
# print(_ns)
# ax.plot(_ns, (_ns - 2) / 4, label='Lewis')

# ax.plot(_ns, _ns / 6 * np.tan(math.pi / 6) / np.tan(math.pi / _ns), label='Quadr. Lewis')

# plt.savefig(f"{figure_path}/Lewis_law.svg")

In [ ]:
# fig, ax = plt.subplots(1, 1)

# data = HC.loc[HC['Experiment'] == 'control']
# data = data.loc[data['stage'] == 'E14']

# groubby_n = data.groupby(by=['Experiment', 'stage', 'position', 'sample_id', 'frame_nb', 'num_neighbors'])
# # groubby_n.apply(lambda d: print(d[['frame_nb', 'num_neighbors', 'normalized_area_cells']]))
# A_n = groubby_n.apply(lambda grp: grp['normalized_area_cells'].mean())
# A_n = A_n.rename('A_n').reset_index().sort_values(by=['Experiment', 'stage', 'position', 'sample_id'], )
# print(A_n)

# sns.scatterplot(
#     data=A_n,
#     x='num_neighbors',
#     y='A_n',
#     hue='stage',
#     hue_order=stage_ordering,
#     style='position',
#     style_order=position_ordering,
#     # size='sample_id',
#     palette=palette_stages,
#     # markers=True,
#     # ls='',
#     # dashes=False,
#     # ci='sd',
#     # err_style='bars'
# )
# sns.regplot(
#     data=A_n,
#     x='num_neighbors',
#     y='A_n',
#     scatter=False
# )
# sns.regplot(
#     data=A_n,
#     x='num_neighbors',
#     y='A_n',
#     scatter=False,
#     order=2
# )

# ax.legend(bbox_to_anchor=(1.01, 1.05))
# ax.set_title("HC")

# _ns = np.array([i for i in range(2, 14)])
# plt.plot(_ns, (_ns - 2) / 4)

# plt.plot(_ns, _ns / 6 * np.tan(math.pi / 6) / np.tan(math.pi / _ns))

# plt.savefig(f"{figure_path}/lewis_law_HC.svg")

In [ ]:
# fig, ax = plt.subplots(1, 1)

# data = SC.loc[SC['Experiment'] == 'control']
# data = data.loc[data['stage'] == 'E14']

# groubby_n = data.groupby(by=['Experiment', 'stage', 'position', 'sample_id', 'frame_nb', 'num_neighbors'])
# # groubby_n.apply(lambda d: print(d[['frame_nb', 'num_neighbors', 'normalized_area_cells']]))
# A_n = groubby_n.apply(lambda grp: grp['normalized_area_cells'].mean())
# A_n = A_n.rename('A_n').reset_index().sort_values(by=['Experiment', 'stage', 'position', 'sample_id'], )
# print(A_n)

# sns.scatterplot(
#     data=A_n,
#     x='num_neighbors',
#     y='A_n',
#     hue='stage',
#     hue_order=stage_ordering,
#     style='position',
#     style_order=position_ordering,
#     # size='sample_id',
#     palette=palette_stages,
#     # markers=True,
#     # ls='',
#     # dashes=False,
#     # ci='sd',
#     # err_style='bars'
# )

# ax.legend(bbox_to_anchor=(1.01, 1.05))
# ax.set_title("SC")

# _ns = np.array([i for i in range(2, 14)])
# print(_ns)
# plt.plot(_ns, (_ns - 2) / 4)

# plt.plot(_ns, _ns / 6 * np.tan(math.pi / 6) / np.tan(math.pi / _ns))

# plt.savefig(f"{figure_path}/lewis_law_SC.svg")

In [ ]:
_HC = HC.loc[HC['Experiment'] == 'control']
_HC = _HC.loc[_HC['stage'] == 'E10']
_HC = _HC.loc[_HC['position'] == '25I']

_SC = SC.loc[SC['Experiment'] == 'control']
_SC = _SC.loc[_SC['stage'] == 'E10']
_SC = _SC.loc[_SC['position'] == '25I']

print("Control")
print("HC norm area: {:.2f}".format(_HC['normalized_area_cells'].mean()))
print("SC norm area: {:.2f}".format(_SC['normalized_area_cells'].mean()))
print("Coordination number: {:.1f} (H: {:.2f}, S: {:.1f})".format(
    _HC['num_neighbors'].mean(),
    _HC['num_hair_neighbors'].mean(),
    _HC['num_neighbors'].mean() - _HC['num_hair_neighbors'].mean()))
print("Subordination number: {:.1f} (H: {:.1f}, S: {:.1f})".format(
    _SC['num_neighbors'].mean(),
    _SC['num_hair_neighbors'].mean(),
    _SC['num_neighbors'].mean() - _SC['num_hair_neighbors'].mean()))

_hc = HC.loc[HC['Experiment'] == 'ML7']
_hc = _hc.loc[_hc['stage'] == 'E10']
_hc = _hc.loc[_hc['position'] == '25I']

_sc = SC.loc[SC['Experiment'] == 'ML7']
_sc = _sc.loc[_sc['stage'] == 'E10']
_sc = _sc.loc[_sc['position'] == '25I']
print("\nML-7")
print("HC norm area: {:.2f}".format(_hc['normalized_area_cells'].mean()))
print("SC norm area: {:.2f}".format(_sc['normalized_area_cells'].mean()))
print("Coordination number: {:.1f} (H: {:.2f}, S: {:.1f})".format(
    _hc['num_neighbors'].mean(),
    _hc['num_hair_neighbors'].mean(),
    _hc['num_neighbors'].mean() - _hc['num_hair_neighbors'].mean()))
print("Subordination number: {:.1f} (H: {:.1f}, S: {:.1f})".format(
    _sc['num_neighbors'].mean(),
    _sc['num_hair_neighbors'].mean(),
    _sc['num_neighbors'].mean() - _sc['num_hair_neighbors'].mean()))

In [ ]:
fig, ax = plt.subplots(1, 1)

sns.regplot(data=HC, x='normalized_area_cells', y='num_neighbors',
            scatter=False, color='red')
sns.kdeplot(data=HC, x='normalized_area_cells', y='num_neighbors', alpha=0.3,
            color='orange')
sns.scatterplot(
    data=HC, 
    x='normalized_area_cells',
    y='num_neighbors',
    style='position',
    style_order=position_ordering,
    hue='stage',
    hue_order=stage_ordering,
    palette=palette_stages,
    alpha=0.4,
    legend=False
)

sns.lineplot(data=HC, x='HC_normalized_area', y='num_neighbors',
            **lineplot_position_stage_kwargs)

# ax.legend(bbox_to_anchor=(1.01, 1.05))

ax.set_xlim([0., 6])
ax.legend(fontsize='x-small')

plt.savefig(f"{figure_path}/num_neighbors__regplo.svg")

In [ ]:
fig, ax = plt.subplots(1, 1)

data = cells.groupby(by=['stage', 'position', 'SI_position', 'PD_position', 'file_id'])
data = data.mean()[['HC_normalized_area', 'is_HC']]
data = data.reset_index()

data = data.loc[data['stage'] != 'E13']
data['SI_position_long'] = data['SI_position'].where(data['SI_position'] != 'S', 'Superior')
data['SI_position_long'] = data['SI_position_long'].where(data['SI_position'] != 'None', 'Mid')
data['SI_position_long'] = data['SI_position_long'].where(data['SI_position'] != 'I', 'Inferior')

data['Stage'] = data['stage']
data['S-I position'] = data['SI_position_long']
data['P-D position'] = data['PD_position']

sns.scatterplot(
  data=data,
  x='HC_normalized_area', y='is_HC',
  hue='Stage',
  hue_order=stage_ordering,
  palette=palette_stages,
  #  style='position',
  #  style_order=position_ordering,
  style='P-D position',
  size='S-I position',
  size_order=['Superior', 'Mid', 'Inferior'],
  #  **lineplot_kwargs
  # markers=True,
  # ls='',
  # dashes=False,
  ci=None,
)

ax.set_xlim([0., 6])
ax.set_ylim([0., 0.4])
ax.set_xlabel('Normalised HC area')
ax.set_ylabel('HC density')

data = cells.groupby(by='file_id')
_area = data['normalized_area_cells'].mean()
_area_std = data['normalized_area_cells'].std()

print(data.mean())

ax.errorbar(
  x=HC.groupby(by='file_id')['normalized_area_cells'].mean(),
  y=data['is_HC'].mean(),
  xerr=HC.groupby(by='file_id')['normalized_area_cells'].std(),
  yerr=data['is_HC'].count()**0.5/data['is_HC'].count(),
  ls='', color='black', 
  linewidth=0.2, alpha=0.5,
  zorder=0
)

plt.legend(prop={'size': 'small'})

# plt.savefig(f"{figure_path}/contacts_vs_area__explants.svg")

In [ ]:
fig, ax = plt.subplots(1, 1)
sns.lineplot(data=HC, x='HC_normalized_area', y='hexatic_order',        
              **lineplot_position_stage_kwargs)
ax.set_ylim([0., 1.])

ax.set_xlim([0., 6])
ax.legend(fontsize='x-small')


plt.savefig(f"{figure_path}/hexatic_order.svg")

In [ ]:
fig, ax = plt.subplots(1, 1)

sns.lineplot(data=HC, x='HC_normalized_area', y='shape_index',
             **lineplot_position_stage_kwargs)
_lineplot_position_stage_kwargs = lineplot_position_stage_kwargs.copy()
_lineplot_position_stage_kwargs['palette'] = 'cool'
sns.lineplot(data=SC, x='HC_normalized_area', y='shape_index', 
             **_lineplot_position_stage_kwargs)
ax.set_ylabel("Shape index")

ax.legend(bbox_to_anchor=(1.01, 1.05))

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(np.linspace(HC['HC_normalized_area'].min(),
                    HC['HC_normalized_area'].max() + 1, 101),
        np.linspace(HC['HC_normalized_area'].min(),
                    HC['HC_normalized_area'].max() + 1, 101), c='black')
sns.lineplot(data=HC, x='HC_normalized_area', y='normalized_area_cells',
             **lineplot_position_stage_kwargs)
ax.set_ylabel("HC area")

ax.legend(bbox_to_anchor=(1.01, 1.05))

## PCP

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': 'polar'})

print(bonds.loc[bonds['type'] == 'SS']['bond_orientation'].max())
sns.histplot(data=bonds.loc[bonds['type'] == 'SS'], ax=ax,
             x='bond_orientation', binrange=[0, 6.2832], bins=32,
             hue='position', multiple='stack',
             stat='density')


In [ ]:
fig, ax = plt.subplots(1, 1)#, subplot_kw={'projection': 'polar'})

print(bonds.loc[bonds['type'] == 'SS']['bond_orientation'].max())
sns.histplot(data=bonds.loc[bonds['type'] == 'SS'], ax=ax,
             x='bond_orientation', y='bond_length_in_px', binrange=[[0, math.pi], [0, 60]],# bins=32,
             bins=8, 
            #  hue='position', multiple='stack',
             stat='density')

## Empirical laws

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, sharex=True)

sns.lineplot(data=SC.loc[SC['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette='cool', ax=ax1, markers=True, dashes=False, ls='', err_style='bars')
sns.lineplot(data=HC.loc[HC['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette=palette_stages, ax=ax2, markers=True, dashes=False, ls='', err_style='bars')
sns.lineplot(data=cells.loc[cells['stage'] == 'E10'], x='num_neighbors', y='normalized_area_cells', hue='stage', style='position', ci='sd', palette=palette_stages, ax=ax3, markers=True, dashes=False, ls='', err_style='bars')

 
ax1.legend(bbox_to_anchor=(2.8, 1.05))
ax2.legend(bbox_to_anchor=(2.2, 1.05))


plt.savefig(f"{figure_path}/A_n.svg")

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.hist(HC['num_neighbors'], align='left', bins=range(3, 15))
ax.set_xlabel("n")
ax.set_ylabel("coordination number")

ax.set_xlim(2, 12)

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.hist(SC['num_hair_neighbors'], align='left', bins=range(0, 12))
ax.set_xlabel("n")
ax.set_ylabel("SC to HC contacts")

ax.set_xlim(0, 10)

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.hist(SC['num_neighbors'] - SC['num_hair_neighbors'],
        align='left', bins=range(0, 12))
ax.set_xlabel("n")
ax.set_ylabel("SC t0 SC contacts")

ax.set_xlim(0, 10)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)

ax1.hist(HC['area_cells'], align='left')
ax1.set_ylabel("HC")

ax2.hist(SC['area_cells'], align='left')
ax2.set_ylabel("SC")
ax2.set_ylim([0, 1])

ax3.hist(cells['area_cells'], align='left')
ax3.set_xlabel("area")
ax3.set_ylabel("all")

print(SC.loc[SC['sum_px_intensity_cells_ch1'] > 0.5e6][['local_id_cells', 'center_x_cells', 'center_y_cells']])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)

ax1.hist(cells['s0_stretch_cells'], align='mid', bins=30)
ax1.set_ylabel("s0 stretch")
ax2.hist(cells['s1_stretch_cells'], align='mid', bins=30)
ax2.set_ylabel("s1 stretch")
ax3.hist(cells['s2_stretch_cells'], align='mid', bins=30)
ax3.set_ylabel("s2 stretch")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)

sns.histplot(data=cells, x='orientation_cells', hue='position', stat='probability', multiple='stack', common_norm=False, binrange=[0., math.pi], ax=ax1)
sns.histplot(data=cells.loc[cells['position'] == '50I'], x='orientation_cells', hue='position', stat='probability', multiple='stack', common_norm=False, binrange=[0., math.pi], ax=ax2)

ax1.axvline(math.pi / 2, c='black', linestyle=':')
ax2.axvline(math.pi / 2, c='black', linestyle=':')

## Polarity

In [ ]:
g = sns.FacetGrid(
    HC,
    row='stage',
    row_order=stage_ordering,
    col="position",
    col_order=position_ordering,
    # col_order=['25I', '75I'],
    hue='sample_id',
    # sharex='row',
    sharey=False,
    subplot_kws=dict(projection='polar'),
    despine=False
)
g.map_dataframe(
    sns.histplot,
    x="cilium_phi_corrected",
    binrange=[-math.pi, math.pi],
    bins=np.linspace(-math.pi, math.pi, 11),
    multiple='stack',
    weights="cilium_rho_corrected"
)


plt.savefig(f"{figure_path}/polarity_histplot.svg")

In [ ]:
g = sns.FacetGrid(
    HC,
    row='stage',
    row_order=stage_ordering,
    hue="position",
    # col_order=position_ordering,
    # hue_order=['25I', '75I'],
    sharex=False,
    sharey=False,
    despine=False
)
g.map_dataframe(
    sns.histplot,
    x="cilium_phi_corrected",
    weights="cilium_rho_corrected",
    binrange=[-math.pi, math.pi],
    bins=12,
)
g.set_axis_labels(x_var='', y_var='')
g.add_legend()

In [ ]:
results = pd.DataFrame()

_data = cells.loc[cells['is_HC']].copy()
_data = _data.loc[(_data['aligned_with_PD_axis']) | (_data['stage'] == 'E14')].copy()
_data = _data.loc[(_data['position'] == '25I') | (_data['position'] == '75I')].copy()

fig, axes = plt.subplots(
    1, 4,
    sharex=True, sharey=True,
    subplot_kw=dict(projection='polar'),
    figsize=[7., 3],
)
plt.subplots_adjust(hspace=0.)
(ax1, ax2, ax3, ax4) = axes
for i, (stage, ax) in enumerate(zip(['E8', 'E10', 'E12', 'E14'], axes)):
    plt.sca(ax)
    data = _data.loc[_data['stage'] == stage].copy()
    
    palette = ['tab:blue', 'tab:orange']
    __data = data
    tmp = data
    tmp['cilium_phi_corrected'] = data['cilium_phi_corrected'] + 2*math.pi
    __data = pd.concat([__data, tmp])
    tmp['cilium_phi_corrected'] = data['cilium_phi_corrected'] - 2*math.pi
    __data = __data.loc[__data['cilium_phi_corrected'] > - 1.2 * math.pi]
    __data = __data.loc[__data['cilium_phi_corrected'] <   1.2 * math.pi]
    __data = pd.concat([__data, tmp])
    sns.kdeplot(
        data=__data.reset_index(),
        x='cilium_phi_corrected',
        y='cilium_rho_corrected',
        hue='position',
        hue_order=['25I', '75I'],
        palette=palette,
        # bw_method=0.1,
        common_norm=False,
        fill=True,
        levels=3,
        thresh=0.2,
        alpha=0.4,
        zorder=10,
    )
    
    sns.scatterplot(
        data=data,
        x='cilium_phi_corrected',
        y='cilium_rho_corrected',
        hue='position',
        hue_order=['25I', '75I'],
        palette=palette,
        # style='file_id',
        alpha=0.75,
        s=3,
        edgecolors=None,
        ax=ax,
        zorder=11,
    )

    # sns.histplot(
    #     data=data,
    #     x='cilium_phi_corrected',
    #     weights='cilium_rho_corrected',
    #     hue='position',
    #     hue_order=['25I', '75I'],
    #     multiple='stack',
    #     bins=16,
    #     binrange=[-math.pi, math.pi],
    #     stat='probability',
    #     zorder=20,
    # )
    
    quiver_x = []
    quiver_y = []
    quiver_u = []
    quiver_v = []
    for pos in np.unique(data['position']):
        d = data.loc[data['position'] == pos]
        mean = circular_mean(d[['cilium_DX_corrected', 'cilium_DY_corrected']], eulerian=True)
        mean_length = circular_mean_length(d[['cilium_DX_corrected', 'cilium_DY_corrected']], eulerian=True, normalize=False)

        quiver_x.append(0)
        quiver_y.append(0)
        quiver_u.append(mean)
        quiver_v.append(mean_length)

    ax.quiver(
        quiver_x, quiver_y, quiver_u, quiver_v,
        facecolor=palette, edgecolor='black', linewidth=linewidth,
        scale=1, angles='xy', scale_units='xy',
        zorder=12
    )

    ax.invert_xaxis()
    ax.set_xlim(-math.pi, math.pi)
    ax.set_xticks([-math.pi, -math.pi / 2, 0, math.pi / 2])
    ax.set_xticklabels(['P', 'I', 'D', 'S'])
    ax.tick_params(axis='x', which='major', pad=-5)

    ax.set_ylim(0, 1.2)
    ax.set_yticks([0.5, 1.])

    ax.set_xlabel('')
    ax.set_ylabel('')

    ax.grid(zorder=-100)

    if i == len(axes) - 1:
        mpl.pyplot.legend(bbox_to_anchor=(1.255,1.25))
    else:
        ax.get_legend().remove()

plt.savefig(f"{figure_path}/polarity.svg")
        

# human readable information

grp = ['stage', 'position', 'file_id']
polarity_by_sample = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_sample['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_sample['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_sample['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_sample['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_sample['circular_mean__angles'] = (polarity_by_sample['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_sample['circular_stddev__angles'] = (polarity_by_sample['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_sample.to_string())

grp = ['stage', 'position']
polarity_by_position = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_position['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_position['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_position['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_position['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_position['circular_mean__angles'] = (polarity_by_position['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_position['circular_stddev__angles'] = (polarity_by_position['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_position.to_string())

grp = ['stage']
polarity_by_stage = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].mean()
polarity_by_stage['circular_mean'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean, eulerian=True)
polarity_by_stage['circular_mean_length'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_mean_length, eulerian=True, normalize=False)
polarity_by_stage['circular_stddev'] = _data.groupby(by=grp)[['cilium_DX_corrected', 'cilium_DY_corrected']].apply(circular_stddev, eulerian=True)
polarity_by_stage['datapoints'] = _data.groupby(by=grp)['cilium_DX_corrected'].apply(lambda d: d.count())

polarity_by_stage['circular_mean__angles'] = (polarity_by_stage['circular_mean']/math.pi * 180 + 360) % 360
polarity_by_stage['circular_stddev__angles'] = (polarity_by_stage['circular_stddev']/math.pi * 180 + 360) % 360
print(polarity_by_stage.to_string())


In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=[1.4, 1.2])

# def position(stage: str):
#     if stage == 'E8':
#         return 0
#     if stage == 'E10':
#         return 1

#     if stage == 'E12':
#         return 2

#     if stage == 'E14':
#         return 3


# for stage, d in results_tissue.iterrows():
#     ax.quiver(
#         0.15*position(stage),
#         # 0,
#         0,
#         d['circular_mean_length'] * np.cos(d['circular_mean']),
#         d['circular_mean_length'] * np.sin(d['circular_mean']),
#         scale=1, angles='xy', scale_units='xy',
#     )

#     ax.text(0.15*position(stage), 0.1, stage, ha='center')

# ax.set_aspect(1)

# ax.set_xlim([-.2, 0.7])
# ax.set_xlabel("P-D axis")


# ax.set_ylim([-.5, 0.2])
# ax.set_yticks([0.2, 0., -0.2, -0.4])
# ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(2))

# ax.hlines([0], xmin=-0.2, xmax=0.7, color='black', lw=0.1)

# ax.set_ylabel("S-I axis")

# ax.set_title('Global polarity')

# plt.savefig(f"{figure_path}/global_polarity.svg")

In [ ]:
treatments = dict({
    'MLCK': [
        'MLCK',
        'MLCK_16hrs',
    ],
    'ROCK': [
        'ROCK',
        'ROCK_16hrs',
    ],
    'PTX': [
        'PTX'
    ],
    'Atoh1': [
        'Atoh1'
    ],
    'Vangl2': [
        'Vangl2'
    ]
})

figsizes=dict({
    'ROCK': 1.6,
    'Atoh1': 1.8,
})
# for Mode in ['aligned']:
for Mode in ['original', 'aligned', 'by_file']:
    if Mode == 'by_file':
        mean_by_file=True
    else:
        mean_by_file=False

    for treatment, items in treatments.items():
        if treatment in figsizes:
            figsize=figsizes[treatment]
        else:
            figsize=1.4
        if len(items) > 1:
            fig, axes = plt.subplots(
                1, len(items),
                sharex=True, sharey=True,
                subplot_kw=dict(projection='polar'),
                figsize=[len(items)*figsize, figsize],
            )

            plt.subplots_adjust(hspace=0.)
        elif len(items) == 1:
            fig, ax = plt.subplots(
                1, 1,
                sharex=True, sharey=True,
                subplot_kw=dict(projection='polar'),
                figsize=[figsize, figsize],
            )
            axes = [ax]

        for i, ax in enumerate(axes):
            plt.sca(ax)
            data = polarity_data.loc[polarity_data['Experiment_Series']==items[i]].copy()
            data = data.sort_values(by='is_Control')

            N = data.groupby(by='is_Control')['cilium_phi_corrected'].count()
            markersize=3
            markersize_control=markersize
            markersize_inhibition=markersize
            if N.iloc[0] > 5 * N.iloc[1]:
                markersize_control *= 2
            elif 5 * N.iloc[0] > N.iloc[1]:
                markersize_inhibition *= 2

            # correct 
            if Mode == 'aligned':
                phi_column='cilium_phi_corrected__aligned'
                rho_column='cilium_rho_corrected'
                dx_column='cilium_DX_corrected__aligned'
                dy_column='cilium_DY_corrected__aligned'

            else:
                phi_column='cilium_phi_corrected'
                rho_column='cilium_rho_corrected'
                dx_column='cilium_DX_corrected'
                dy_column='cilium_DY_corrected'

            palette = ['tab:blue', 'tab:red'] # ['navy', 'orange']
            hue_order=[False, True]
            style=None
            if Mode == 'by_file':
                style='file_id'
            sns.scatterplot(
                data=data,
                x=phi_column,
                y=rho_column,
                hue='is_Control',
                hue_order=hue_order,
                size='is_Control',
                size_order=[True, False],
                sizes=[markersize_control, markersize_inhibition],
                style=style,
                palette=palette,
                alpha=0.75,
                # s=3,
                edgecolors=None,
                ax=ax,
                zorder=11,
            )
            __data = data.copy()

            tmp = data.loc[data[phi_column] < -1.2 * math.pi].copy()
            tmp[phi_column] = tmp[phi_column] + 2*math.pi
            __data = pd.concat([__data, tmp])

            tmp = data.loc[data[phi_column] > 1.2 * math.pi].copy()
            tmp[phi_column] = tmp[phi_column] - 2*math.pi
            __data = pd.concat([__data, tmp])

            sns.kdeplot(
                data=__data.reset_index(),
                x=phi_column,
                y=rho_column,
                hue='is_Control',
                hue_order=hue_order,
                palette=palette,
                fill=True,
                common_norm=False,
                levels=3,
                thresh=.2,
                alpha=0.4,
                zorder=10,
            )
            
            quiver_x = []
            quiver_y = []
            quiver_u = []
            quiver_v = []
            color = []
            for j, is_control in enumerate(hue_order):
                d = data.loc[data['is_Control'] == is_control]

                # if treatment == 'Vangl2':
                if mean_by_file:
                    mean_phi = d.groupby(by='file_id')[[dx_column, dy_column]].apply(circular_mean, eulerian=True)
                    mean_rho = d.groupby(by='file_id')[[dx_column, dy_column]].apply(circular_mean_length, eulerian=True, normalize=False)

                else:
                    mean_phi = [circular_mean(d[[dx_column, dy_column]], eulerian=True)]
                    mean_rho = [circular_mean_length(d[[dx_column, dy_column]], eulerian=True, normalize=False)]
                
                for phi, rho in zip(mean_phi, mean_rho):
                    quiver_x.append(0)
                    quiver_y.append(0)
                    quiver_u.append(phi)
                    quiver_v.append(rho)
                    color.append(palette[j])

            ax.quiver(
                quiver_x, quiver_y, quiver_u, quiver_v,
                facecolor=color, edgecolor='black', linewidth=linewidth,
                scale=1, angles='xy', scale_units='xy',
                zorder=12
            )

            ax.set_xlim(-math.pi, math.pi)
            ax.set_xticks([-math.pi, -math.pi / 2, 0, math.pi / 2])
            ax.set_xticklabels(['P', 'I', 'D', 'S'])
            ax.tick_params(axis='x', which='major', pad=-5)

            ax.set_ylim(0, 1.2)
            ax.set_yticks([0.5, 1.])

            ax.set_xlabel('')
            ax.set_ylabel('')

            if i == len(axes) - 1:
                leg_handles = ax.get_legend_handles_labels()[0]
                mpl.pyplot.legend(
                    leg_handles[::-1],
                    ['Control', f'{treatment}'],
                    title='',
                    fontsize=ticksize,
                    bbox_to_anchor=(1.05,1.05),
                    markerscale=1
                )
            else:
                ax.get_legend().remove()

        plt.savefig(f"{figure_path}/polarity__{treatment}__{Mode}.svg")

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw=dict(projection='polar'))
sns.scatterplot(
    data=cells_experiments.loc[(cells_experiments['Experiment_Series']=='Vangl2') & (cells_experiments['is_Control'])],
    x='cilium_phi_corrected',
    y='cilium_rho_corrected',
    hue='is_Control',
    style='file_id',
    alpha=0.75,
    s=3,
    edgecolors=None,
    ax=ax,
    zorder=0,
)

# Atoh1

In [ ]:

fix, ax = plt.subplots(1, 1, figsize=[1.2, 1.7])

sns.violinplot(
    data=__tissue_data,
    x='Treatment',
    order=['Atoh1control', 'Atoh1KO'],
    palette=['tab:red', 'tab:blue'],
    y='HC_density',
    inner='point',
    linewidth=linewidth
)

box_pairs=[
    ('Atoh1control', 'Atoh1KO'),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=__tissue_data,
    x='Treatment',
    order=['Atoh1control', 'Atoh1KO'],
    y='HC_density',
    split=False,
)
text_format='star'
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
    loc='outside',
    text_format=text_format
)
annotator.apply_and_annotate()


ax.set_xlabel('')
ax.set_xticklabels(['Control', 'Atoh1-KO'], rotation=30, ha='right', rotation_mode='anchor')

ax.set_ylabel('HC density')
ax.set_ylim(0, 0.4)
ax.set_yticks([0, 0.2, 0.4])

plt.savefig(f"{figure_path}/HC_density_Atoh1.svg")

## Available sets

In [ ]:
print("Available datasets: ")
for set in cells.columns:
  print("  {}".format(set))

In [ ]:
print("Available datasets in bonds: ")
for set in bonds.columns:
  print("  {}".format(set))

## Atoh1

In [ ]:
atoh1_cells = cells_experiments.loc[cells_experiments['Experiment_Series'] == 'Atoh1']
atoh1_bonds = bonds_experiments.loc[bonds_experiments['Experiment_Series'] == 'Atoh1']
print(np.unique(atoh1_cells['filename']))

atoh1Control_cells = atoh1_cells.loc[atoh1_cells['is_Control']]
atoh1KO_cells = atoh1_cells.loc[~atoh1_cells['is_Control']]
print(np.unique(atoh1Control_cells['filename']))
print(np.unique(atoh1KO_cells['filename']))

print()
atoh1_tissues = __tissue_data.loc[__tissue_data['Experiment_Series'] == 'Atoh1']
print(np.unique(atoh1_tissues['filename']))


In [ ]:
fix, ax = plt.subplots(1, 1)

sns.violinplot(
    data=atoh1_cells,
    # data=__tissue_data.loc[__tissue_data['stage'] == 'E10'],
    x='Treatment',
    order=['Control', 'Atoh1control', 'Atoh1KO'],
    hue='is_HC',
    y='normalized_area_cells',
    inner='point',
    linewidth=0.25
)


# box_pairs=[
#     ('Atoh1control', 'Atoh1KO'),
#     ('Control', 'Atoh1control'),
#     ('Control', 'Atoh1KO'),
# ]

# annotator = Annotator(
#     ax,
#     box_pairs,
#     # data=atoh1_tissues,
#     data=__tissue_data.loc[__tissue_data['stage'] == 'E10'],
#     x='Treatment',
#     order=['Control', 'Atoh1control', 'Atoh1KO'],
#     # order=['Atoh1control', 'Atoh1KO'],
#     y='HC_density',
#     split=False,
# )
# annotator.configure(
#     test='Mann-Whitney',
#     line_width=0.25,
#     loc='outside',
# )
# annotator.apply_and_annotate()



# ax.set_xlabel('')
# ax.set_xticklabels(['Fixed BP', 'Control', 'Atoh1-KO'], rotation=45, ha='right', rotation_mode='anchor')

# ax.set_ylabel('HC density')
# ax.set_ylim(0, 0.4)
# ax.set_yticks([0, 0.2, 0.4])



In [ ]:
fix, axes = plt.subplots(1, 5, figsize=[16, 4])

for ax, column in zip(axes, ['HC (all)', 'HC-HC', 'SC (all)', 'SC-HC', 'SC-SC']):
    if column == 'HC (all)':
        data = cells_experiments.loc[cells_experiments['is_HC']]
        __column = 'num_neighbors__corr'
    elif column == 'HC-HC':
        data = cells_experiments.loc[cells_experiments['is_HC']]
        __column = 'num_hair_neighbors'
    elif column == 'SC (all)':
        data = cells_experiments.loc[~cells_experiments['is_HC']]
        __column = 'num_neighbors__corr'
    elif column == 'SC-HC':
        data = cells_experiments.loc[~cells_experiments['is_HC']]
        __column = 'num_hair_neighbors'
    elif column == 'SC-SC':
        data = cells_experiments.loc[~cells_experiments['is_HC']]
        data['num_support_neighbors'] = data['num_neighbors__corr'] - data['num_hair_neighbors']
        __column = 'num_support_neighbors'
    else:
        continue
    sns.violinplot(
        data=data,
        x='Treatment',
        order=['Atoh1control', 'Atoh1KO'],
        y=__column,
        inner='quartile',
        linewidth=0.5,
        bw=0.1,
        ax=ax
    )
    box_pairs=[
        ('Atoh1control', 'Atoh1KO'),
    ]

    annotator = Annotator(
        ax,
        box_pairs,
        data=data,
        x='Treatment',
        order=['Atoh1control', 'Atoh1KO'],
        y=__column,
        split=False,
    )
    annotator.configure(
        test='Mann-Whitney',
        line_width=0.25,
        # loc='outside',
    )
    annotator.apply_and_annotate()

    ax.set_xlabel('')

    ax.set_ylabel(column)
    ax.set_ylim(0, 12)
    ax.set_yticks(range(0, 14, 2))
    ax.set_xticklabels(['Atoh1 Control', 'Atoh1-KO'], rotation=45, ha='right', rotation_mode='anchor')

In [ ]:
for num_neighbors, description in zip(['num_neighbors__corr', 'num_neighbors'], ['__corrected', '']):
    fig, ax = plt.subplots(1, 1, figsize=[2., 2.], sharex=True, sharey=True)

    for axis in ['bottom','left']:
        ax.spines[axis].set_linewidth(1./4)
    for axis in ['top','right']:
        ax.spines[axis].set_linewidth(0)
    ax.tick_params(which='both', width=1./4)

    stages = [
    'E10',
    # 'E10',
    # 'E10',
    # 'E10',
    ]
    position = '25I'

    y_min = 0
    y_max=10
    y_ticks = np.arange(0, y_max+1)
    y_tick_labels = [
    '0',# '',
    '3',# '',
    '6',# '',
    '9',# '',
    # '12',
    ]


    histograms = dict()
    means = dict()
    for Experiment in ['Atoh1control', 'Atoh1KO']:
        data = cells_experiments.loc[
            (cells_experiments['position'] == position) &
            (cells_experiments['Experiment'] == Experiment)
        ]

        hc = data.loc[data['is_HC']]
        sc = data.loc[~data['is_HC']]


        # HC neighborhood
        hc_norm = hc.groupby(by='stage')[num_neighbors].count()

        _hist = [hc.loc[(hc[num_neighbors] > i - 0.5) & (hc[num_neighbors] < i + 0.5)].groupby(by='stage')[num_neighbors].count() for i in y_ticks]
        _hist = pd.concat(_hist, axis=1, keys=y_ticks)
        _hist = _hist.divide(hc_norm, axis=0)
        _hist['type'] = 'H'

        hist = _hist

        ### HH
        _hist = [hc.loc[hc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
        _hist = pd.concat(_hist, axis=1, keys=y_ticks)
        _hist = _hist.divide(hc_norm, axis=0)
        _hist['type'] = 'HH'

        hist = pd.concat([hist, _hist], axis=0, join='outer')

        ### HS
        hc['num_support_neighbors'] = hc[num_neighbors] - hc['num_hair_neighbors']
        _hist = [hc.loc[(hc['num_support_neighbors'] > i - 0.5) & (hc['num_support_neighbors'] < i + 0.5)].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
        _hist = pd.concat(_hist, axis=1, keys=y_ticks)
        _hist = _hist.divide(hc_norm, axis=0)
        _hist['type'] = 'HS'

        hist = pd.concat([hist, _hist], axis=0, join='outer')


        # SC neighborhood
        sc_norm = sc.groupby(by='stage')[num_neighbors].count()

        _hist = [sc.loc[(sc[num_neighbors] > i - 0.5) & (sc[num_neighbors] < i + 0.5)].groupby(by='stage')[num_neighbors].count() for i in y_ticks]
        _hist = pd.concat(_hist, axis=1, keys=y_ticks)
        _hist = _hist.divide(sc_norm, axis=0)
        _hist['type'] = 'S'

        hist = pd.concat([hist, _hist], axis=0, join='outer')

        # SH
        _hist = [sc.loc[sc['num_hair_neighbors'] == i].groupby(by='stage')['num_hair_neighbors'].count() for i in y_ticks]
        _hist = pd.concat(_hist, axis=1, keys=y_ticks)
        _hist = _hist.divide(sc_norm, axis=0)
        _hist['type'] = 'SH'

        hist = pd.concat([hist, _hist], axis=0, join='outer')

        # SS
        sc['num_support_neighbors'] = sc[num_neighbors] - sc['num_hair_neighbors']
        _hist = [sc.loc[(sc['num_support_neighbors'] > i - 0.5) & (sc['num_support_neighbors'] < i + 0.5)].groupby(by='stage')['num_support_neighbors'].count() for i in y_ticks]
        _hist = pd.concat(_hist, axis=1, keys=y_ticks)
        _hist = _hist.divide(sc_norm, axis=0)
        _hist['type'] = 'SS'


        # concat
        hist = pd.concat([hist, _hist], axis=0, join='outer')
        hist = hist.reset_index()
        hist = hist.set_index(['stage', 'type'])

        hist = hist.reset_index()
        histograms[Experiment] = hist

        _means = []
        mean_data = data
        tmp = [
            mean_data.loc[mean_data['is_HC']][num_neighbors].mean(),
            mean_data.loc[mean_data['is_HC']]['num_hair_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']][num_neighbors].mean(),
            mean_data.loc[~mean_data['is_HC']]['num_hair_neighbors'].mean(),
            mean_data.loc[~mean_data['is_HC']][num_neighbors].mean() - mean_data.loc[~mean_data['is_HC']]['num_hair_neighbors'].mean(),
        ]
        _means.append(tmp)
        means[Experiment] = _means

    x_ticks = [
    'H',
    'HH',
    # 'HS',
    'S',
    'SH',
    'SS'
    ]
    x_tick_labels = [
    'HC (all)',
    'HC-HC',
    #'', 'HC-SC',
    'SC (all)', 
    'SC-HC',
    'SC-SC',
    ]


    set_colors = [
    'red',
    'orangered',
    # 'gold',
    'dimgrey',
    'darkgray',
    'silver'
    ]

    set_background = [
    # 'coral',
    # '',
    # 'gray',
    ]


    histplot_violin_col(
        hist=histograms['Atoh1control'],
        col_column='stage',
        col_labels=stages,
        col_axes=[ax],
        x_ticks=x_ticks,
        x_tick_labels=x_tick_labels,
        y_ticks=y_ticks,
        y_tick_labels=y_tick_labels,
        set_colors=set_colors,
        data_column='type',
        set_background=set_background,
        hist_split=histograms['Atoh1KO'],
        # means=means,
        means=means['Atoh1control'],
        means_split=means['Atoh1KO'],
    )


    # plt.savefig(f"{figure_path}/neighborhood__Atoh1{description}.svg")

# Circular statistics

In [ ]:
def randomvector(n):
    components = [np.random.normal() for i in range(n)]
    r = math.sqrt(sum(x*x for x in components))
    v = [x/r for x in components]
    return v

print(randomvector(2))
def randomangle():
    v = randomvector(2)
    return np.arctan2(v[1], v[0])



In [ ]:
sns.histplot(
    data=[randomangle() for i in range(10000)]
)

In [ ]:
mean = 0.2 + math.pi / 2
# scale = 0.01
# scale = 0.1
scale = math.pi / 6
length = 0.1
std = 0.2

In [ ]:
def random_angles(mean, scale, N):
    return np.random.normal(loc=math.pi + mean, scale=scale, size=N) % (2*math.pi) - math.pi
    # return np.random.uniform(low=mean-scale, high=mean+scale,size=N)

def random_radii(mean, scale, N):
    return np.random.lognormal(np.log(mean), scale, N)

fig, (ax1, ax2) = plt.subplots(1, 2)
sns.histplot(
    data=random_angles(mean, scale, 5000),
    binrange=[-math.pi, math.pi],
    ax=ax1
)
sns.histplot(
    data=random_radii(length, std, 5000),
    ax=ax2
)

In [ ]:
N = 5000

data = pd.DataFrame()
data['phi'] = random_angles(mean, scale, N)
data['rho'] = random_radii(length, std, N)
data['X'] = data['rho'] * np.cos(data['phi'])
data['Y'] = data['rho'] * np.sin(data['phi'])

circ_stddev = circular_stddev(data[['phi']], eulerian=False)
print('{:.2f}, {:.3f}, {:.5f}, {:.5f}'.format(
    circular_mean(data[['X', 'Y']], eulerian=True),
    circular_variance(data[['X', 'Y']], eulerian=True),
    circular_stddev(data[['X', 'Y']], eulerian=True),
    circ_stddev
))

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': 'polar'})
sns.scatterplot(
    data=data,
    x='phi',
    y='rho',
    s=3,
)
sns.kdeplot(
    data=data,
    x='phi',
    y='rho',
    levels=3,
    alpha=0.4,
    zorder=-1
)

m = circular_mean(data[['X', 'Y']], eulerian=True)
R = circular_mean_length(data[['X', 'Y']], eulerian=True)
print(m, R, circular_mean_length(data[['X', 'Y']], eulerian=True, normalize=True), circular_variance(data[['X', 'Y']], eulerian=True) )
ax.quiver(
    0., 0., m, R,
    scale=1, angles='xy', scale_units='xy'
)
ax.set_xlim([-math.pi, math.pi])
# ax.set_ylim([0, 1.2])